# SHAKEuq: Uncertainty Quantification and Bayesian Analysis of SHAKEmap Products

This notebook presents the **uncertainty quantification (UQ) framework**
implemented within the `SHAKEuq` class of the SHAKEmaps Toolkit.

It focuses on analyzing how uncertainty associated with SHAKEmap products
**evolves across versions**, how this uncertainty can be **interpreted on a
unified spatial grid**, and how different **updating strategies** may be applied
to refine uncertainty estimates in rapid-response contexts.

The notebook is designed as an **executable uncertainty analysis workflow**,
combining product inspection, sanity checks, target-based diagnostics, and
map-based visualization of uncertainty behavior.

---

## Overview

SHAKEmap products do not only evolve in terms of median shaking estimates (e.g.,
MMI, PGA), but also include **spatially varying uncertainty** that reflects
limitations in data coverage, modeling assumptions, and epistemic knowledge
throughout the response timeline.

The SHAKEuq uncertainty framework provides a structured approach to:

- extract uncertainty information directly from SHAKEmap products,
- normalize uncertainty fields across versions using a unified grid,
- analyze uncertainty evolution at points, areas, and global extents,
- compare published (RAW) uncertainty against predicted or updated uncertainty,
- and visualize both temporal and spatial uncertainty changes.

This notebook emphasizes **uncertainty-focused analysis**, rather than
ground-motion modeling itself. Median shaking values are treated as inputs,
while uncertainty behavior is the primary object of study.

---

## Core Functionality

The workflows demonstrated in this notebook include:

- Parsing SHAKEmap uncertainty products (`uncertainty.xml`)
- Unified-grid construction for cross-version uncertainty comparison
- Version-to-version uncertainty evolution diagnostics
- Point, area, and global uncertainty decay analysis
- Bayesian and hierarchical uncertainty updating strategies
- Sigma change and reduction mapping
- Sanity checks and RAW-vs-unified validation tools

---

## Intended Use

This notebook is intended for:

- research-oriented uncertainty analysis,
- uncertainty-aware rapid-response decision support,
- sensitivity and robustness studies of SHAKEmap products,
- development and testing of uncertainty updating methodologies,
- validation of uncertainty propagation assumptions.

It is **not** intended to replace operational SHAKEmap generation, but to
provide a transparent and reproducible framework for **post-hoc uncertainty
analysis and experimentation**.

---

## Dependencies

This notebook assumes the following Python environment:

`Python 3.x` `numpy` `pandas` `scipy` `matplotlib` `xml (built-in)`

Optional (depending on analysis and plots):
`cartopy` `geopandas`

Custom project modules:
`SHAKEuq` `SHAKEparser` `SHAKEmapper` `SHAKEtools`

---

`Last update: January 2026` `SHAKEuq version: v26.5`


# Event Overview

This section defines the **event context** needed to run SHAKEuq: the event ID/time, local file paths (ShakeMap products, station lists, rupture files), the available **version list**, and the **target definitions** (points/areas). The goal is to make the notebook runnable by changing **only a small set of event parameters** before executing the UQ workflow.


## Notebook Setup  
### Dock files Shakemap 

In [ ]:
from modules.SHAKEparser import *
from modules.SHAKEmapper import *
from modules.SHAKEtools import *
from modules.SHAKEtime import *
from modules.SHAKEuq import *


base_folder = 'event_data/SHAKEfetch'

#Select Event
event_id = 'us7000m9g4'         #'us7000pn9s' Myanmar ; 'us7000m9g4' Taiwan ; 'us6000jllz' Turkey 7.8 ;'us6000jlqa' Turkey 7.5 


In [ ]:

if event_id == 'us7000pn9s': #Myanmar #
    event_id = 'us7000pn9s'
    event_epicenter_lon = 95.925
    event_epicenter_lat = 22.001
    version = '022'
    first_version = '001'
    event_time = '2025-03-28 06:20:52'

    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    version_list = ['001', '002','003', '004','005','006', '007', '008','009', '011', '012','013','014','015','016','017','018','019','020','021','022']
    selected_cities = ['Mandalay', 'Nay Pyi Taw','Chiang Mai' ,'Taungoo','Bangkok']

    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/us7000pn9s_us_1_cdi_geo_1km.txt"
    
    all_dists = (
            'normal',
            'lognorm',
            'gamma',
            'weibull',
            'burr')

    
elif event_id == 'us6000jllz': #Turkey 7.8
    event_id = 'us6000jllz'
    event_epicenter_lon = 37.014
    event_epicenter_lat = 37.226
    version = '017'
    first_version = '001'

    event_time = '2023-02-06 01:17:34'
    
    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    
    version_list = ['001', '002','003', '004','006', '007', '008','009', '011', '012']
    version_list = ['001', '002','003', '004','005','006', '007', '008','009', '010','011', '012','013','014','015','016','017']
    selected_cities = ['Elbistan','Gaziantep', 'Sanliurfa','Kahramanmaras' ,'Kilis','Narli','Malatya','Diyarbakir','Antakya' ,'Adana','Aleppo','Elazig','Iskenderun']
    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/{event_id}_us_1_cdi_geo_1km.txt"

    all_dists = ( #Taiwan
        'normal',
        'lognorm',
        'gamma',
        'beta',
        'genextreme',
        'pearson3',
        'burr')

elif event_id == 'us6000jlqa': #Turkey 7.5
    event_id = 'us6000jlqa'
    event_epicenter_lon = 37.196
    event_epicenter_lat = 38.011
    version = '016'
    first_version = '001'

    event_time = '2023-02-06 10:24:48'
    
    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    
    version_list = ['001', '002','003', '004','005','006', '007', '008','009', '010','011', '012','013','014','015','016']
    selected_cities = ['Elbistan','Gaziantep', 'Sanliurfa','Kahramanmaras' ,'Kilis','Narli','Malatya','Diyarbakir','Antakya' ,'Adana','Aleppo','Elazig','Iskenderun']
    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/{event_id}_us_1_cdi_geo_1km.txt"

    all_dists = (
        'normal',
        'lognorm',
        'gamma',
        'beta',
        'genextreme',
        'pearson3',
        'burr')


elif event_id == 'us7000m9g4': #Taiwan

    event_id = 'us7000m9g4'
    version = '014'
    first_version = '001'
    event_epicenter_lon = 121.598
    event_epicenter_lat = 23.835


    
    xml_shakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{version}_grid.xml"
    xml_firstshakemap_file_path = f"./{base_folder}/usgs-shakemap-versions/{event_id}/{event_id}_us_{first_version}_grid.xml"

    file_path_rupturedata = f'./{base_folder}/usgs-rupture-versions/{event_id}/{event_id}_us_{version}_rupture.json'  # Replace with actual file path
    
    file_path_instrument_data = f"./{base_folder}/usgs-instruments_data-versions/{event_id}/{event_id}_us_{version}_stationlist.json"
    file_path_dyfixml = f"./{base_folder}/usgs-dyfi-versions/{event_id}/{event_id}_us_3_cdi_geo_1km.txt"

    event_time = '2024-04-02 23:58:12'
    
    shakemap_folder = "./event_data/SHAKEfetch/usgs-shakemap-versions"
    pager_folder = "./event_data/SHAKEfetch/usgs-pager-versions"
    rupture_folder = "./event_data/SHAKEfetch/usgs-rupture-versions"
    stations_folder = "./event_data/SHAKEfetch/usgs-instruments_data-versions"
    dyfi_xml = "./event_data/SHAKEfetch/usgs-dyfi-versions/us7000m9g4/us7000m9g4_us_3_cdi_geo_1km.txt"

    version_list = ['001', '002','003', '004','006', '007', '008','009', '011', '012','013','014']
    
    selected_cities = ['Hualien City', 'Yilan', 'Banqiao','Taipei' ,'Taoyuan','Keelung']

    all_dists = ( #Taiwan
            'normal',
            'lognorm',
            'gamma',
            'weibull',
            'expon',
            'beta',
            'genextreme',
            'pearson3',
            'burr')



### Target Definistions

In [ ]:
# ============================================================
# TARGET DEFINITIONS — USER INPUTS
# ============================================================

# Choose which cities to use (IDs must be unique and stable for filenames/plots)
# You can reuse your selected_cities list, but here we define explicit coordinates.
#
# NOTE: Replace coordinates if you prefer a different reference (city center, station, etc.)


points_city = []

if event_id == "us7000m9g4":  # Taiwan example set
    points_city = [
        {"id": "Taipei",    "lat": 25.0330, "lon": 121.5654},
        {"id": "Hualien",   "lat": 23.9872, "lon": 121.6015},
        {"id": "TW_Point_01", "lat": 24.9770, "lon": 120.9240},
        {"id": "Kaohsiung",   "lat": 22.6273, "lon": 120.3014},
    ]

elif event_id == "us7000pn9s":  # Myanmar example set
    points_city = [
        {"id": "Mandalay",   "lat": 21.9588, "lon": 96.0891},
        {"id": "NayPyiTaw",  "lat": 19.7633, "lon": 96.0785},
    ]

elif event_id in ("us6000jllz", "us6000jlqa"):  # Turkey example set
    points_city = [
        {"id": "Gaziantep",      "lat": 37.0662, "lon": 37.3833},
        {"id": "Kahramanmaras",  "lat": 37.5753, "lon": 36.9371},
        {"id": "Antakya", "lat": 36.2021, "lon": 36.1606},
        {"id": "Malatya", "lat": 38.3552, "lon": 38.3095},
    ]

else:
    # Fallback: epicenter as a “city-like” point if event not in presets
    points_city = [
        {"id": "Epicenter", "lat": float(event_epicenter_lat), "lon": float(event_epicenter_lon)}
    ]

print("[OK] Point targets:")
for p in points_city:
    print(" ", p)



In [ ]:


# ============================================================
# AREA TARGETS — CITY-CENTERED RADII
# ============================================================

# Two radii to test (km) — adjust per your study design
radius_km_small = 10.0
radius_km_large = 25.0

areas_city = []

for p in points_city:
    # Example A: small radius around city
    areas_city.append({
        "id": f"{p['id']}_R{int(radius_km_small)}km",
        "type": "circle",
        "lat": float(p["lat"]),
        "lon": float(p["lon"]),
        "radius_km": float(radius_km_small),
    })

    # Example B: larger radius around same city
    areas_city.append({
        "id": f"{p['id']}_R{int(radius_km_large)}km",
        "type": "circle",
        "lat": float(p["lat"]),
        "lon": float(p["lon"]),
        "radius_km": float(radius_km_large),
    })

print("[OK] Area targets:")
for a in areas_city:
    print(" ", a)

# ============================================================
# TARGET PACKS (for modular plotting blocks)
# ============================================================

# Pack 1: points only (city centers)
targets_points = {
    "points": points_city,
    "areas": None,
    "global_stat": None,
}

# Pack 2: areas only (circles around cities)
targets_areas = {
    "points": None,
    "areas": areas_city,
    "global_stat": None,
}

# Pack 3 (optional): combine points + areas in the same calls (supported by your plotters)
targets_points_and_areas = {
    "points": points_city,
    "areas": areas_city,
    "global_stat": None,
}

print("[OK] Target packs ready: targets_points, targets_areas, targets_points_and_areas")


# ============================================================
# REPRODUCIBILITY ECHO (recommended)
# ============================================================

target_manifest = {
    "event_id": event_id,
    "points_city": points_city,
    "areas_city": areas_city,
    "radius_km_small": radius_km_small,
    "radius_km_large": radius_km_large,
}

target_manifest



In [ ]:
from modules.SHAKEtime import *
from modules.SHAKEuq import *

# Smoke Test Code Update 

In [ ]:
def smoke_uq_v26p8_cdi(
    event_id,
    event_time,
    shakemap_folder,
    pager_folder,
    dyfi_xml,
    stations_folder=None,
    rupture_folder=None,
    version_list=None,
    points_city=None,
    IMT="MMI",
    quick_versions=10,
    which="sigma_total_predicted",
    methods=("ShakeMap", "bayes", "bayes_2lik", "dyfi_weighted"),
    update_radius_km=25.0,
    kernel="gaussian",
    kernel_scale_km=20.0,
    measurement_sigma=0.30,
    measurement_sigma_instr=0.20,
    measurement_sigma_dyfi=0.50,
    dyfi_use_after_hours=24.0,
    dyfi_cdi_max_dist_km=400.0,
    sigma_total_from_shakemap=True,
    sigma_aleatory=0.40,
    UQ_BASE_FOLDER="./export",
    OUT_ROOT="./export",
    export_reports=True,
    PRINT=True,
    N_HEAD=8,
):
    import numpy as np
    import pandas as pd
    from pathlib import Path
    from modules.SHAKEuq import SHAKEuq

    imtU = str(IMT).upper().strip()
    if version_list is None:
        raise ValueError("version_list is required.")
    vlist = sorted([int(v) for v in version_list])[: int(quick_versions)]
    if len(vlist) < 2:
        raise ValueError("need at least 2 versions.")

    if not points_city:
        raise ValueError("points_city is required.")

    v_first, v_last = vlist[0], vlist[-1]

    out_dir = Path(OUT_ROOT) / "SHAKEuq" / str(event_id) / "uq" / "smoke_v26p8_cdi"
    out_dir.mkdir(parents=True, exist_ok=True)

    if PRINT:
        print("\n" + "=" * 76)
        print("[SMOKE v26.8] START — CDI-enabled validation")
        print(f"  event_id     : {event_id}")
        print(f"  IMT          : {imtU}")
        print(f"  versions     : {v_first:03d} → {v_last:03d} (n={len(vlist)})")
        print(f"  methods      : {methods}")
        print(f"  σ(base)      : {measurement_sigma}")
        print(f"  σ(instr/dyfi): {measurement_sigma_instr} / {measurement_sigma_dyfi}")
        print(f"  DYFI modes   : stationlist vs auto(>= {dyfi_use_after_hours}h) vs cdi")
        print(f"  CDI file     : {dyfi_xml}")
        print(f"  OUT          : {out_dir}")
        print("=" * 76)

    shake = SHAKEuq(
        event_id,
        event_time,
        shakemap_folder,
        pager_folder,
        file_type=2,
        stations_folder=stations_folder,
        rupture_folder=rupture_folder,
        dyfi_cdi_file=dyfi_xml,
        dyfi_use_after_hours=float(dyfi_use_after_hours),
        dyfi_cdi_max_dist_km=float(dyfi_cdi_max_dist_km),
        dyfi_source="stationlist",
    )

    if PRINT:
        print("[SMOKE] uq_build_dataset ...")
    Path(UQ_BASE_FOLDER).mkdir(parents=True, exist_ok=True)
    shake.uq_build_dataset(
        event_id=event_id,
        version_list=vlist,
        base_folder=UQ_BASE_FOLDER,
        stations_folder=stations_folder,
        rupture_folder=rupture_folder,
        imts=(imtU,),
        grid_unify="intersection",
        resolution="finest",
        export=True,
        interp_method="nearest",
        interp_kwargs=None,
    )
    if PRINT:
        print("[SMOKE] uq_state built ✅")

    # targets (2 pts)
    pts = []
    for p in list(points_city)[:2]:
        pp = dict(p)
        pp.setdefault("type", "point")
        pp["id"] = str(pp.get("id", pp.get("name", "point")))
        pp["lat"] = float(pp["lat"])
        pp["lon"] = float(pp["lon"])
        pts.append(pp)

    # --- audit obs under each dyfi_source mode ---
    def _audit_obs(dyfi_source_tag):
        rows = []
        for v in vlist:
            obs, c = shake._uq_collect_obs_for_version(
                int(v),
                imtU,
                measurement_sigma=float(measurement_sigma),
                include_weights=True,
                prefer_domain=False,
                allow_fallback=True,
                measurement_sigma_instr=measurement_sigma_instr,
                measurement_sigma_dyfi=measurement_sigma_dyfi,
                attach_per_obs_sigma=True,
                dyfi_source=str(dyfi_source_tag),
            )

            mv = []
            for o in obs:
                if isinstance(o, dict) and o.get("meas_var", None) is not None:
                    try:
                        mv.append(float(o["meas_var"]))
                    except Exception:
                        pass
            mv = np.asarray(mv, float) if mv else np.asarray([], float)

            rows.append(
                dict(
                    run=str(dyfi_source_tag),
                    version=int(v),
                    n_obs=int(c.get("total", 0)),
                    n_seismic=int(c.get("seismic", 0)),
                    n_intensity=int(c.get("intensity", 0)),
                    n_meas_var=int(np.isfinite(mv).sum()),
                    meas_var_min=(float(np.nanmin(mv)) if mv.size else np.nan),
                    meas_var_med=(float(np.nanmedian(mv)) if mv.size else np.nan),
                    meas_var_max=(float(np.nanmax(mv)) if mv.size else np.nan),
                )
            )
        return pd.DataFrame(rows)

    if PRINT:
        print("[SMOKE] obs audits (stationlist / auto / cdi) ...")
    df_obs = pd.concat(
        [_audit_obs("stationlist"), _audit_obs("auto"), _audit_obs("cdi")],
        ignore_index=True,
    )

    if PRINT:
        print(df_obs.head(N_HEAD))
        try:
            auto_n = df_obs[df_obs["run"] == "auto"][["version", "n_intensity"]].set_index("version")["n_intensity"]
            sta_n  = df_obs[df_obs["run"] == "stationlist"][["version", "n_intensity"]].set_index("version")["n_intensity"]
            if (auto_n != sta_n).any():
                print("✅ [SMOKE] AUTO differs from stationlist for some versions (CDI switch likely active).")
            else:
                print("⚠️  [SMOKE] AUTO matches stationlist for all versions (either <24h, missing timestamps, or CDI not used).")
        except Exception:
            pass

    # --- extraction under each mode (set attribute, no signature change) ---
    def _extract(tag, dyfi_source_tag):
        old = getattr(shake, "dyfi_source", "stationlist")
        shake.dyfi_source = str(dyfi_source_tag)
        df = shake.uq_extract_target_series(
            version_list=vlist,
            imt=imtU,
            points=pts,
            areas=None,
            agg="mean",
            prior_version=v_first,
            sigma_total_from_shakemap=sigma_total_from_shakemap,
            sigma_aleatory=sigma_aleatory,
            update_radius_km=float(update_radius_km),
            kernel=str(kernel),
            kernel_scale_km=float(kernel_scale_km),
            measurement_sigma=float(measurement_sigma),
            measurement_sigma_instr=float(measurement_sigma_instr),
            measurement_sigma_dyfi=float(measurement_sigma_dyfi),
            methods_to_compute=methods,
            audit=True,
            audit_output_path=str(out_dir),
            audit_prefix=f"{tag}-{imtU}",
        )
        shake.dyfi_source = old
        df = df.copy()
        df["dyfi_source"] = str(dyfi_source_tag)
        df["smoke_tag"] = str(tag)
        return df

    if PRINT:
        print("[SMOKE] running curve extraction (stationlist / auto / cdi) ...")
    df_sta = _extract("stationlist_TEST", "stationlist")
    df_aut = _extract("auto_TEST", "auto")
    df_cdi = _extract("cdi_TEST", "cdi")

    # divergence stats for bayes_2lik (sigma curve)
    df_cmp = None
    if "bayes_2lik" in methods:
        def _p(df):
            d = df[df["method"] == "bayes_2lik"][["target_id", "version", which]].copy()
            return d.rename(columns={which: "val"})
        p_sta = _p(df_sta).rename(columns={"val": "sta"})
        p_aut = _p(df_aut).rename(columns={"val": "auto"})
        p_cdi = _p(df_cdi).rename(columns={"val": "cdi"})
        df_cmp = p_sta.merge(p_aut, on=["target_id", "version"], how="inner").merge(p_cdi, on=["target_id", "version"], how="inner")
        df_cmp["auto_minus_station"] = df_cmp["auto"] - df_cmp["sta"]
        df_cmp["cdi_minus_station"] = df_cmp["cdi"] - df_cmp["sta"]

        if PRINT:
            a = df_cmp["auto_minus_station"].values
            c = df_cmp["cdi_minus_station"].values
            a = a[np.isfinite(a)]
            c = c[np.isfinite(c)]
            print("[SMOKE] bayes_2lik divergence stats:")
            print("  auto - stationlist:", pd.Series(a).describe() if a.size else "(none)")
            print("  cdi  - stationlist:", pd.Series(c).describe() if c.size else "(none)")

    if export_reports:
        df_obs.to_csv(out_dir / f"smoke_obs_{imtU}_station_auto_cdi.csv", index=False)
        df_sta.to_csv(out_dir / f"smoke_curves_stationlist_{imtU}.csv", index=False)
        df_aut.to_csv(out_dir / f"smoke_curves_auto_{imtU}.csv", index=False)
        df_cdi.to_csv(out_dir / f"smoke_curves_cdi_{imtU}.csv", index=False)
        if df_cmp is not None:
            df_cmp.to_csv(out_dir / f"smoke_cmp_bayes2lik_station_auto_cdi_{imtU}.csv", index=False)

    if PRINT:
        print("[SMOKE v26.8] DONE ✅")
        print("  saved to:", out_dir)
        print("=" * 76 + "\n")

    return {
        "out_dir": str(out_dir),
        "df_obs": df_obs,
        "df_stationlist": df_sta,
        "df_auto": df_aut,
        "df_cdi": df_cdi,
        "df_cmp": df_cmp,
    }


In [ ]:
dyfi_xml = "./event_data/SHAKEfetch/usgs-dyfi-versions/us7000m9g4/us7000m9g4_us_3_cdi_geo_1km.txt"

result = smoke_uq_v26p8_cdi(
    event_id=event_id,
    event_time=event_time,
    shakemap_folder=shakemap_folder,
    pager_folder=pager_folder,
    stations_folder=stations_folder,
    rupture_folder=rupture_folder,
    version_list=version_list,
    points_city=points_city,
    dyfi_xml=dyfi_xml,
    IMT="MMI",
    quick_versions=10,
    PRINT=True,
)


In [ ]:
shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)


shake.debug_uq = True  # optional, prints meas_var usage

df = shake.uq_plot_targets_decay(
    version_list=vlist,
    imt="MMI",
    points=[{"id":"CITY_A","type":"point","lat":36.8,"lon":34.6}],
    areas=None,
    what="sigma_total_predicted",
    methods=("ShakeMap","bayes","bayes_2lik","dyfi_weighted"),
    prior_version=vlist[0],
    measurement_sigma=0.30,
    measurement_sigma_instr=0.20,
    measurement_sigma_dyfi=0.50,
    output_path="./export",
    save=True,
    show=False,
)


In [ ]:
df_obs, df_ctrl, df_test, df_cmp = smoke_uq_v26p6(
    IMT="MMI",
    quick_versions=12,
    measurement_sigma=0.30,
    measurement_sigma_instr=0.20,
    measurement_sigma_dyfi=0.50,
    make_area_test=False,
    PRINT=True,
)



In [ ]:
FIGSIZE_data_plot = (6,4)
FIGSIZE_single_map = (6.2,4.5)
FIGSIZE_paired_maps = (6,6)
FIGSIZE_full_page = (6,8)

FIGDPIs =300
SAVE_FORMATSs = ['pdf', 'png']


LABEL_SIZEs = 17
TICKS_SIZEs = 17
TITLE_SIZEs = 7
LEGEND_SIZEs = 9
LINEWIDTHs  = 1
MARKER_STYLEs = "o"
MARKER_SIZEs = 5
LEGEND_KWARGs = {"loc": "best", "frameon": True}

target_point_uq_decay =  [ 'MMI']  # False or ['PGA'] ['MMI'] ['PGA', 'MMI']


from modules.SHAKEparser import *
from modules.SHAKEmapper import *
from modules.SHAKEtools import *
from modules.SHAKEtime import *

# END OF SMOKE TEST 

In [ ]:
import sys 
sys.exit()

### Behaviour Settings 


In [ ]:
run_sanity_checks = True

#uncertainty_quantification = False 
#target_uq_decay = True


target_point_uq_decay =  [ 'MMI']  # False or ['PGA'] ['MMI'] ['PGA', 'MMI']
target_area_uq_decay = ['MMI'] # False or ['PGA'] ['MMI'] ['PGA', 'MMI']
target_global_decay = ['MMI'] # False or ['PGA'] ['MMI'] ['PGA', 'MMI']

run_decay_suite = False # True or False 

run_mean_suite = [ 'MMI']  # False or ['PGA'] ['MMI'] ['PGA', 'MMI']




plot_sigma_maps = [ 'MMI'] # False or ['PGA'] ['MMI'] ['PGA', 'MMI']
raw_vs_unified_validation = False # True or False 






In [ ]:
# FIGIS

### Figure Size

| Figure type                  | Figsize (in)   | DPI | Use                    |
| ---------------------------- | -------------- | --- | ---------------------- |
| Data plots (2×2)             | **(6, 4)**     | 300 | Time series, residuals |
| Single SHAKEmap              | **(6.2, 4.5)** | 300 | Half-page map          |
| Paired uncertainty maps      | **(6, 6)**     | 300 | MMI / PGA              |
| 2×2 SHAKEmaps                | **(6.4, 6.4)** | 300 | Model comparison       |
| Large decay grid (full page) | **(6.5, 9.0)** | 300 | n×m summary            |
| Small support figure         | **(4, 3)**     | 300 | Methods only           |


In [ ]:
from modules.SHAKEparser import *
from modules.SHAKEmapper import *
from modules.SHAKEtools import *
from modules.SHAKEtime import *



import os

In [ ]:
print('Section Cleared!')

# UQ Framework Version + Product Inspection

This section performs **sanity checks and product inspection** to confirm the framework is reading the expected ShakeMap assets correctly. You’ll verify:
- which IMTs are available per version,
- whether `grid.xml` and `uncertainty.xml` are present and consistent,
- that the unified grid build controls behave as intended,
- and that outputs are being written to the expected `.../SHAKEuq/<event_id>/uq/` structure.

Think of this section as the **“does everything load and align correctly?”** checkpoint before running decay plots and maps.


## Shared output + version normalization

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd



if run_sanity_checks:
    # -----------------------------
    # Shared output folder
    # -----------------------------
    out_root = "./export"
    Path(out_root).mkdir(parents=True, exist_ok=True)
    
    # -----------------------------
    # Version normalization
    # -----------------------------
    version_list_int = [int(v) for v in version_list]
    v_first = int(version_list_int[0])
    v_final = int(version_list_int[-1])
    
    print("[OK] event_id =", event_id)
    print("[OK] versions =", v_first, "→", v_final, "| n =", len(version_list_int))
    print("[OK] out_root =", out_root)
    
    
    shake = SHAKEtime(event_id, event_time, shakemap_folder, pager_folder, file_type=2)

    
    
    print("[OK] SHAKEuq initialized")
    print("  shakemap_folder =", shakemap_folder)
    print("  pager_folder    =", pager_folder)
    print("  rupture_folder  =", rupture_folder)
    print("  stations_folder =", stations_folder)



## Version inspection tables (summary + IMTs) (code)

In [ ]:
if run_sanity_checks:  

    # --- Summary table (metadata per version) ---
    summary_list = shake.get_shake_summary(version_list=version_list, shakemap_folder=shakemap_folder)
    df_summary = shake.get_dataframe(summary_list)

    display(df_summary.head(10))
    print("[OK] df_summary rows =", len(df_summary))

    # --- IMT availability per version ---
    try:
        imts_by_version = shake.uq_list_available_imts(version_list=version_list_int)
        df_imts = (
            pd.DataFrame({
                "version": list(imts_by_version.keys()),
                "n_imts": [len(v) for v in imts_by_version.values()],
                "imts": [", ".join(v) for v in imts_by_version.values()],
            })
            .sort_values("version")
            .reset_index(drop=True)
        )
        display(df_imts)
        print("[OK] IMT availability table built")
    except Exception as e:
        print("[WARN] uq_list_available_imts failed:", repr(e))


## UQ build controls (user knobs)

In [ ]:
if run_sanity_checks:
    shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)

    # ============================================================
    # UQ DATASET BUILD (Unified Grid) — USER KNOBS
    # ============================================================
    
    # Core: choose IMTs you want in the unified dataset
    uq_imts = ("MMI", "PGA", "PGV", "PSA")     # you can add "PSA03" etc. if the event provides it
    
    # Unified grid policy
    uq_grid_unify = "intersection"            # recommended default for fair cross-version comparison
    uq_resolution = "finest"                  # "finest" is canonical in your patch
    
    # Interpolation policy used when mapping each version to the unified grid
    uq_interp_method = "nearest"              # safest for RAW-vs-UNIFIED validation
    uq_interp_kwargs = None                   # usually None unless testing (e.g., {"fill_value": np.nan})
    
    # Export control (recommended True for publish-ready reproducibility)
    uq_export = True
    
    # Where dataset gets written
    uq_base_folder = "./export"     # will create ./export/SHAKEuq/<event_id>/uq/
    
    print("[UQ CONFIG]")
    print("  IMTs         =", uq_imts)
    print("  grid_unify   =", uq_grid_unify)
    print("  resolution   =", uq_resolution)
    print("  interp       =", uq_interp_method)
    print("  export       =", uq_export)
    print("  base_folder  =", uq_base_folder)


## Unified Grid Construction

In [ ]:
%time

# ============================================================
# BUILD UQ DATASET (creates shake.uq_state)
# ============================================================

if run_sanity_checks:

    uq_state = shake.uq_build_dataset(
        event_id=event_id,
        version_list=version_list_int,
        base_folder=uq_base_folder,
        stations_folder=stations_folder,
        rupture_folder=rupture_folder,
        imts=uq_imts,
        grid_unify=uq_grid_unify,
        resolution=uq_resolution,
        export=uq_export,
        interp_method=uq_interp_method,
        interp_kwargs=uq_interp_kwargs,
        output_units=None,   # optional: {"PGA":"%g"} etc. if you want enforced output units
    )

    print("[OK] UQ dataset built")
    print("  uq_state keys:", list(uq_state.keys()))
    print("  base_folder   :", shake.uq_state.get("base_folder"))
    print("  interp_method :", shake.uq_state.get("interp_method"))


## Sanity table + minimal “examiner checks”

In [ ]:
# ============================================================
# SANITY REPORT (Examiner-friendly QA table)
# ============================================================

if run_sanity_checks:

    df_sanity = shake.uq_sanity_report()
    display(df_sanity)

    # -----------------------------
    # Minimal hard checks (fail early)
    # -----------------------------
    assert hasattr(shake, "uq_state") and shake.uq_state is not None, "uq_state missing — uq_build_dataset failed."

    # How many versions actually made it into the unified dataset?
    built_versions = sorted(list(shake.uq_state.get("versions_unified", {}).keys()))
    print("[OK] versions_unified:", built_versions[:5], "...", built_versions[-5:])
    print("[OK] n_versions_unified =", len(built_versions))

    # Quick file availability counts (useful when events have missing uncertainty.xml or stations)
    if isinstance(df_sanity, pd.DataFrame):
        cols = [c for c in df_sanity.columns if c.lower() in (
            "version",
            "uncertainty_xml_exists",
            "stationlist_exists",
            "rupture_loaded",
            "n_instrumented",
            "n_dyfi"
        )]
        if cols:
            display(df_sanity[cols])

        # Count missing uncertainty.xml (critical for RAW σ workflows)
        if "uncertainty_xml_exists" in df_sanity.columns:
            n_missing_unc = int((~df_sanity["uncertainty_xml_exists"].astype(bool)).sum())
            print("[QA] missing uncertainty.xml versions =", n_missing_unc)

        # Count missing stationlists
        if "stationlist_exists" in df_sanity.columns:
            n_missing_st = int((~df_sanity["stationlist_exists"].astype(bool)).sum())
            print("[QA] missing stationlist.json versions =", n_missing_st)


In [ ]:
# ============================================================
# OPTIONAL: Define "analysis-ready" versions (recommended)
# ============================================================

if run_sanity_checks:
    
    analysis_versions = built_versions.copy()

    # If sanity report is a DataFrame, we can filter versions more strictly
    if isinstance(df_sanity, pd.DataFrame) and "version" in df_sanity.columns:
        df_s = df_sanity.copy()

        # Example strict policy:
        # - must have grid (already implied)
        # - must have uncertainty.xml for RAW sigma analyses
        # - stationlist optional (depends on what you're doing)
        if "uncertainty_xml_exists" in df_s.columns:
            df_s = df_s[df_s["uncertainty_xml_exists"].astype(bool)]

        analysis_versions = sorted(df_s["version"].astype(int).tolist())

    print("[OK] analysis_versions =", analysis_versions[0], "→", analysis_versions[-1], "| n =", len(analysis_versions))

    # Keep a standard variable name for downstream blocks
    version_list_uq = analysis_versions


In [ ]:
print('Section Cleared!')

# Target UQ — Uncertainty Decay (City Point vs City Radius)

This section demonstrates **how uncertainty evolves across versions** at user-defined targets. Using Examples A/B/C (and suite-style runs if enabled), you will generate:
- **Point target** uncertainty decay curves (city locations),
- **Area target** uncertainty decay curves (city radius circles),
- **Global extent** summaries (whole-map statistics and optional RAW min/max bands).

The goal is to validate the UQ workflow behavior over time and compare methods against the published ShakeMap baseline.



## Point target uncertainty decay

In [ ]:
FIGSIZE_data_plot = (6,4)
FIGSIZE_single_map = (6.2,4.5)
FIGSIZE_paired_maps = (6,6)
FIGSIZE_full_page = (6,8)

FIGDPIs =300
SAVE_FORMATSs = ['pdf', 'png']


LABEL_SIZEs = 17
TICKS_SIZEs = 17
TITLE_SIZEs = 7
LEGEND_SIZEs = 9
LINEWIDTHs  = 1
MARKER_STYLEs = "o"
MARKER_SIZEs = 5
LEGEND_KWARGs = {"loc": "best", "frameon": True}

In [ ]:
%%time

plt.style.use('./bins/latex_font.mplstyle')

# ============================================================
# EXAMPLE A (Standalone): POINT TARGET — UNCERTAINTY DECAY
#   - Point target uncertainty decay
#   - Point UQ — Uncertainty Decay (City Point)
#
# PATH POLICY (IMPORTANT):
#   - uq_build_dataset(base_folder=...) writes to:
#       <base_folder>/<event_id>/uq/
#   - plot/audit helpers (output_path=...) write to:
#       <output_path>/SHAKEuq/<event_id>/uq/
#
# To ALWAYS keep UQ data under:
#   ./export/SHAKEuq/<event_id>/uq
# we set:
#   UQ_BASE_FOLDER = "./export/SHAKEuq"   (dataset)
#   OUT_ROOT       = "./export"            (plots/audits)
# ============================================================

if target_point_uq_decay:
    for imt in target_point_uq_decay:

        from modules.SHAKEtime import SHAKEtime
        from pathlib import Path
        import numpy as np
        import matplotlib.pyplot as plt

        # ------------------------------------------------------------
        # USER KNOBS (change only these)
        # ------------------------------------------------------------
        IMT = str(imt).upper()                 # "MMI" or "PGA"
        WHICH = "sigma_total_predicted"        # e.g. "sigma_total_predicted", "sigma_ep_predicted", ...

        # --- IMPORTANT PATHS ---
        UQ_BASE_FOLDER = "./export"  # dataset root -> ./export/SHAKEuq/<event_id>/uq
        OUT_ROOT = "./export"                 # plots/audits root -> ./export/SHAKEuq/<event_id>/uq/...

        SAVE_FORMATS = SAVE_FORMATSs
        DPI = FIGDPIs

        # methods in uq_plot_targets_decay (your API)
        METHODS = ("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo")

        # ------------------------------------------------------------
        # UQ MODEL KNOBS (simple defaults)
        # ------------------------------------------------------------
        sigma_total_from_shakemap = True  # RAW ShakeMap σ_total is baseline

        # Aleatory assumptions (used in decomposition logic, not overwriting RAW σ)
        SIGMA_ALEATORY_MMI = 0.40
        SIGMA_ALEATORY_PGA = 0.35

        def sigma_aleatory_for_imt(imtU):
            imtU0 = str(imtU).upper().strip()
            if imtU0 == "MMI":
                return SIGMA_ALEATORY_MMI
            if imtU0 in ("PGA", "PGV"):
                return SIGMA_ALEATORY_PGA
            return None

        # Update model knobs
        update_radius_km = 25.0
        kernel = "gaussian"
        kernel_scale_km = 20.0

        MEAS_SIGMA_MMI = 0.30
        MEAS_SIGMA_PGA = 0.35

        def measurement_sigma_for_imt(imtU):
            imtU0 = str(imtU).upper().strip()
            if imtU0 == "MMI":
                return MEAS_SIGMA_MMI
            if imtU0 == "PGA":
                return MEAS_SIGMA_PGA
            return 0.30

        # Kriging knobs
        ok_range_km = 60.0
        ok_variogram = "exponential"
        ok_nugget = 1e-6
        ok_sill = None
        ok_cap_sigma_to_prior = True

        # Monte Carlo knobs
        mc_nsim = 500
        mc_include_aleatory = True

        # Unified grid knobs
        grid_res = None
        interp_method = "nearest"
        interp_kwargs = None

        # ------------------------------------------------------------
        # PLOT STYLE (top-level, easy tuning)
        # ------------------------------------------------------------
        FIGSIZE = FIGSIZE_data_plot
        COMBINED_FIGSIZE = FIGSIZE_full_page
        xrotation = 0
        ylog = False
        ymin = None
        ymax = None

        label_size  = LABEL_SIZEs
        tick_size   = TICKS_SIZEs
        title_size  = TITLE_SIZEs
        legend_size = LEGEND_SIZEs

        linewidth = LINEWIDTHs
        marker_style = MARKER_STYLEs
        markersize = MARKER_SIZEs
        legend_kwargs = LEGEND_KWARGs

        # Optional: your mplstyle
        # plt.style.use('./bins/latex_font.mplstyle')

        # ------------------------------------------------------------
        # REQUIRED (from your event setup cell)
        # ------------------------------------------------------------
        imtU = str(IMT).upper().strip()
        vlist = sorted([int(v) for v in version_list])
        v_first, v_last = vlist[0], vlist[-1]

        Path(UQ_BASE_FOLDER).mkdir(parents=True, exist_ok=True)
        Path(OUT_ROOT).mkdir(parents=True, exist_ok=True)

        # ------------------------------------------------------------
        # 0) Build a fresh SHAKEuq instance
        # ------------------------------------------------------------
        shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
        shake.stations_folder = stations_folder
        shake.rupture_folder  = rupture_folder

        print("[OK] event:", event_id, "| IMT:", imtU)
        print("[OK] versions:", v_first, "→", v_last, "| n =", len(vlist))

        # ------------------------------------------------------------
        # 1) Build unified dataset (ALWAYS under ./export/SHAKEuq/<event_id>/uq)
        # ------------------------------------------------------------
        base_folder = UQ_BASE_FOLDER
        Path(base_folder).mkdir(parents=True, exist_ok=True)

        shake.uq_build_dataset(
            event_id=event_id,
            version_list=vlist,
            base_folder=base_folder,                 # <-- FIXED policy
            stations_folder=stations_folder,
            rupture_folder=rupture_folder,
            imts=(imtU,),
            grid_unify="intersection",
            resolution="finest",
            export=True,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,
        )
        print("[OK] uq_state built")

        # ------------------------------------------------------------
        # 2) Ensure points schema matches SHAKEuq target parser
        # ------------------------------------------------------------
        points_uq = []
        for p in points_city:
            pp = dict(p)
            pp.setdefault("type", "point")
            pp["id"]  = str(pp.get("id", pp.get("name", "point")))
            pp["lat"] = float(pp["lat"])
            pp["lon"] = float(pp["lon"])
            points_uq.append(pp)

        # ------------------------------------------------------------
        # 3) Footer for title / audit prefix
        # ------------------------------------------------------------
        footer = (
            f"event={event_id} | prior=v{v_first:03d} | "
            f"updateR={update_radius_km:.0f}km | kernel={kernel}({kernel_scale_km:.0f}km) | "
            f"sigma_total=RAW(SM)"
        )

        # ------------------------------------------------------------
        # 4) Run POINT uncertainty decay
        # ------------------------------------------------------------
        df_point_decay = shake.uq_plot_targets_decay(
            version_list=vlist,
            imt=imtU,

            points=points_uq,
            areas=None,

            what=WHICH,
            methods=METHODS,
            agg="mean",
            global_stat=None,
            prior_version=v_first,

            sigma_total_from_shakemap=sigma_total_from_shakemap,
            sigma_aleatory=sigma_aleatory_for_imt(imtU),

            update_radius_km=update_radius_km,
            kernel=kernel,
            kernel_scale_km=kernel_scale_km,
            measurement_sigma=measurement_sigma_for_imt(imtU),

            ok_range_km=ok_range_km,
            ok_variogram=ok_variogram,
            ok_nugget=ok_nugget,
            ok_sill=ok_sill,
            ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

            mc_nsim=mc_nsim,
            mc_include_aleatory=mc_include_aleatory,

            grid_res=grid_res,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,

            figsize=FIGSIZE,
            dpi=DPI,
            ylog=ylog,
            ymin=ymin,
            ymax=ymax,
            xrotation=xrotation,

            show_title=True,
            title=f"{imtU} uncertainty decay @ {{target}} (POINT)\n{footer}",
            show_grid=True,
            legend=True,
            legend_kwargs=legend_kwargs,
            tight=True,

            label_size=label_size,
            tick_size=tick_size,
            title_size=title_size,
            legend_size=legend_size,
            xlabel="version",
            ylabel=f"σ_total ({imtU})",

            linewidth=linewidth,
            markerstyle=marker_style,
            markersize=markersize,

            output_path=OUT_ROOT,                 # <-- FIXED policy
            save=True,
            save_formats=SAVE_FORMATS,
            show=True,

            plot_combined=True,
            combined_figsize=COMBINED_FIGSIZE,
            combined_legend_ncol=2,

            audit=True,
            audit_output_path=OUT_ROOT,            # <-- FIXED policy
            audit_prefix=f"UQ-PointDecay-{imtU}",
        )

        print("[OK] Point target uncertainty decay complete")
        df_point_decay.head(10)


In [ ]:
print('Section Cleared!')

## Area target (radius) uncertainty decay

In [ ]:
%%time

plt.style.use('./bins/latex_font.mplstyle')

# ============================================================
# EXAMPLE B (Standalone): AREA TARGET — UNCERTAINTY DECAY
#   - Area UQ — Uncertainty Decay (City Radius)
#
# PATH POLICY (IMPORTANT):
#   - uq_build_dataset(base_folder=...) writes to:
#       <base_folder>/<event_id>/uq/
#   - plot/audit helpers (output_path=...) write to:
#       <output_path>/SHAKEuq/<event_id>/uq/
#
# To ALWAYS keep UQ data under:
#   ./export/SHAKEuq/<event_id>/uq
# we set:
#   UQ_BASE_FOLDER = "./export/SHAKEuq"   (dataset)
#   OUT_ROOT       = "./export"            (plots/audits)
# ============================================================

if target_area_uq_decay:
    for imt in target_area_uq_decay:

        from modules.SHAKEtime import SHAKEtime
        from pathlib import Path
        import numpy as np
        import matplotlib.pyplot as plt

        # ------------------------------------------------------------
        # USER KNOBS (change only these)
        # ------------------------------------------------------------
        IMT = str(imt).upper()                 # "MMI" or "PGA"
        WHICH = "sigma_total_predicted"

        # --- IMPORTANT PATHS ---
        UQ_BASE_FOLDER = "./export"  # dataset root -> ./export/SHAKEuq/<event_id>/uq
        OUT_ROOT = "./export"                 # plots/audits root -> ./export/SHAKEuq/<event_id>/uq/...

        SAVE_FORMATS = SAVE_FORMATSs
        DPI = FIGDPIs

        METHODS = ("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo")

        # Area definition
        radius_km = 30.0  # e.g. 10, 25, 50

        # ------------------------------------------------------------
        # UQ MODEL KNOBS (same defaults as Example A)
        # ------------------------------------------------------------
        sigma_total_from_shakemap = True

        SIGMA_ALEATORY_MMI = 0.40
        SIGMA_ALEATORY_PGA = 0.35

        def sigma_aleatory_for_imt(imtU):
            imtU0 = str(imtU).upper().strip()
            if imtU0 == "MMI":
                return SIGMA_ALEATORY_MMI
            if imtU0 in ("PGA", "PGV"):
                return SIGMA_ALEATORY_PGA
            return None

        update_radius_km = 25.0
        kernel = "gaussian"
        kernel_scale_km = 20.0

        MEAS_SIGMA_MMI = 0.30
        MEAS_SIGMA_PGA = 0.35

        def measurement_sigma_for_imt(imtU):
            imtU0 = str(imtU).upper().strip()
            if imtU0 == "MMI":
                return MEAS_SIGMA_MMI
            if imtU0 == "PGA":
                return MEAS_SIGMA_PGA
            return 0.30

        ok_range_km = 60.0
        ok_variogram = "exponential"
        ok_nugget = 1e-6
        ok_sill = None
        ok_cap_sigma_to_prior = True

        mc_nsim = 500
        mc_include_aleatory = True

        grid_res = None
        interp_method = "nearest"
        interp_kwargs = None

        # ------------------------------------------------------------
        # PLOT STYLE
        # ------------------------------------------------------------
        FIGSIZE = FIGSIZE_data_plot
        COMBINED_FIGSIZE = FIGSIZE_full_page
        xrotation = 0
        ylog = False
        ymin = None
        ymax = None


        label_size  = LABEL_SIZEs
        tick_size   = TICKS_SIZEs
        title_size  = TITLE_SIZEs
        legend_size = LEGEND_SIZEs

        linewidth = LINEWIDTHs
        marker_style = MARKER_STYLEs
        markersize = MARKER_SIZEs
        
        legend_kwargs = LEGEND_KWARGs

        # Optional: your mplstyle
        # plt.style.use('./bins/latex_font.mplstyle')

        # ------------------------------------------------------------
        # REQUIRED (from your event setup cell)
        # ------------------------------------------------------------
        imtU = str(IMT).upper().strip()
        vlist = sorted([int(v) for v in version_list])
        v_first, v_last = vlist[0], vlist[-1]

        Path(UQ_BASE_FOLDER).mkdir(parents=True, exist_ok=True)
        Path(OUT_ROOT).mkdir(parents=True, exist_ok=True)

        # ------------------------------------------------------------
        # 0) Build a fresh SHAKEuq instance
        # ------------------------------------------------------------
        shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
        shake.stations_folder = stations_folder
        shake.rupture_folder  = rupture_folder

        print("[OK] event:", event_id, "| IMT:", imtU)
        print("[OK] versions:", v_first, "→", v_last, "| n =", len(vlist))

        # ------------------------------------------------------------
        # 1) Build unified dataset (ALWAYS under ./export/SHAKEuq/<event_id>/uq)
        # ------------------------------------------------------------
        base_folder = UQ_BASE_FOLDER
        Path(base_folder).mkdir(parents=True, exist_ok=True)

        shake.uq_build_dataset(
            event_id=event_id,
            version_list=vlist,
            base_folder=base_folder,                 # <-- FIXED policy
            stations_folder=stations_folder,
            rupture_folder=rupture_folder,
            imts=(imtU,),
            grid_unify="intersection",
            resolution="finest",
            export=True,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,
        )
        print("[OK] uq_state built")

        # ------------------------------------------------------------
        # 2) Build areas from points_city (circle targets)
        # ------------------------------------------------------------
        areas_uq = []
        for p in points_city:
            pid = str(p.get("id", p.get("name", "point")))
            areas_uq.append({
                "id": f"{pid}_R{int(radius_km)}km",
                "type": "area",
                "kind": "circle",
                "lat": float(p["lat"]),
                "lon": float(p["lon"]),
                "radius_km": float(radius_km),
            })

        print("[OK] Built", len(areas_uq), "area targets (circle)")

        footer = (
            f"event={event_id} | prior=v{v_first:03d} | "
            f"updateR={update_radius_km:.0f}km | kernel={kernel}({kernel_scale_km:.0f}km) | "
            f"sigma_total=RAW(SM)"
        )

        # ------------------------------------------------------------
        # 3) Run AREA uncertainty decay
        # ------------------------------------------------------------
        df_area_decay = shake.uq_plot_targets_decay(
            version_list=vlist,
            imt=imtU,

            points=None,
            areas=areas_uq,

            what=WHICH,
            methods=METHODS,
            agg="mean",
            global_stat=None,
            prior_version=v_first,

            sigma_total_from_shakemap=sigma_total_from_shakemap,
            sigma_aleatory=sigma_aleatory_for_imt(imtU),

            update_radius_km=update_radius_km,
            kernel=kernel,
            kernel_scale_km=kernel_scale_km,
            measurement_sigma=measurement_sigma_for_imt(imtU),

            ok_range_km=ok_range_km,
            ok_variogram=ok_variogram,
            ok_nugget=ok_nugget,
            ok_sill=ok_sill,
            ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

            mc_nsim=mc_nsim,
            mc_include_aleatory=mc_include_aleatory,

            grid_res=grid_res,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,

            figsize=FIGSIZE,
            dpi=DPI,
            ylog=ylog,
            ymin=ymin,
            ymax=ymax,
            xrotation=xrotation,

            show_title=True,
            title=f"{imtU} uncertainty decay @ {{target}} (AREA circle, R={radius_km:.0f} km)\n{footer}",
            show_grid=True,
            legend=True,
            legend_kwargs=legend_kwargs,
            tight=True,

            label_size=label_size,
            tick_size=tick_size,
            title_size=title_size,
            legend_size=legend_size,
            xlabel="version",
            ylabel=f"σ_total ({imtU})",

            linewidth=linewidth,
            markerstyle=marker_style,
            markersize=markersize,

            output_path=OUT_ROOT,                 # <-- FIXED policy
            save=True,
            save_formats=SAVE_FORMATS,
            show=True,

            plot_combined=True,
            combined_figsize=COMBINED_FIGSIZE,
            combined_legend_ncol=2,

            audit=True,
            audit_output_path=OUT_ROOT,            # <-- FIXED policy
            audit_prefix=f"UQ-AreaDecay-R{int(radius_km)}km-{imtU}",
        )

        print("[OK] Area target uncertainty decay complete")
        df_area_decay.head(10)


In [ ]:
print('Section Cleared!')

## Global Decay 

In [ ]:
%%time

plt.style.use('./bins/latex_font.mplstyle')

# ============================================================
# EXAMPLE C (UPDATED, MODULAR): GLOBAL EXTENT
#   - Global Uncertainty Decay
#   - published RAW min/max band + method mean curves
#
# PATH POLICY (IMPORTANT):
#   - uq_build_dataset(base_folder=...) writes to:
#       <base_folder>/<event_id>/uq/
#   - plot/audit helpers (output_path=...) write to:
#       <output_path>/SHAKEuq/<event_id>/uq/
#
# To ALWAYS keep UQ data under:
#   ./export/SHAKEuq/<event_id>/uq
# we set:
#   UQ_BASE_FOLDER = "./export/SHAKEuq"   (dataset)
#   OUT_ROOT       = "./export"            (plots/audits)
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from modules.SHAKEtime import SHAKEtime


if target_global_decay:



    # ------------------------------------------------------------
    # USER KNOBS (change only these)
    # ------------------------------------------------------------
    IMT_LIST = target_global_decay    # ["MMI"] or ["PGA"] or None/False to skip
    WHICH_SIGMA_TOTAL = True      # baseline policy: RAW ShakeMap sigma_total is the prior

    # --- IMPORTANT PATHS ---
    UQ_BASE_FOLDER = "./export"  # dataset root -> ./export/SHAKEuq/<event_id>/uq
    OUT_ROOT = "./export"                 # plots/audits root -> ./export/SHAKEuq/<event_id>/uq/...

    # Update model knobs (match your other examples)
    update_radius_km = 25.0
    kernel = "gaussian"
    kernel_scale_km = 20.0

    ok_range_km = 60.0
    ok_variogram = "exponential"
    ok_nugget = 1e-6
    ok_sill = None
    ok_cap_sigma_to_prior = True

    mc_nsim = 500
    mc_include_aleatory = True

    # Aleatory assumptions (used in decomposition; should NOT overwrite RAW σ published)
    sigma_aleatory_mmi = 0.40
    sigma_aleatory_pga = 0.35

    def sigma_aleatory_for_imt(imt):
        imtU0 = str(imt).upper().strip()
        if imtU0 == "MMI":
            return float(sigma_aleatory_mmi)
        if imtU0 in ("PGA", "PGV"):
            return float(sigma_aleatory_pga)
        return None

    # Measurement sigma per IMT (example defaults)
    measurement_sigma_mmi = 0.30
    measurement_sigma_pga = 0.35

    def measurement_sigma_for_imt(imt):
        imtU0 = str(imt).upper().strip()
        if imtU0 == "MMI":
            return float(measurement_sigma_mmi)
        if imtU0 == "PGA":
            return float(measurement_sigma_pga)
        return 0.30

    # Plot styling (consistent with your notebooks)
    dpi = FIGDPIs
    combined_figsize = FIGSIZE_data_plot
    ylog = False
    ymin = None
    ymax = None


    label_size  = LABEL_SIZEs
    tick_size   = TICKS_SIZEs
    title_size  = TITLE_SIZEs
    legend_size = LEGEND_SIZEs

    linewidth = LINEWIDTHs
    marker_style = MARKER_STYLEs
    markersize = MARKER_SIZEs

    legend_kwargs = {"loc": "best", "frameon": True}

    # ------------------------------------------------------------
    # REQUIRED from your event setup cell
    # ------------------------------------------------------------
    # event_id, event_time, shakemap_folder, pager_folder
    # stations_folder, rupture_folder, version_list
    # ------------------------------------------------------------

    Path(UQ_BASE_FOLDER).mkdir(parents=True, exist_ok=True)
    Path(OUT_ROOT).mkdir(parents=True, exist_ok=True)

    # normalize version list once
    version_list_uq = sorted([int(v) for v in version_list])
    v_first = int(version_list_uq[0])

    # footer (per IMT we’ll render a slightly different footer)
    def make_footer(imtU):
        return (
            f"event={event_id} | prior=v{v_first:03d} | "
            f"updateR={update_radius_km}km | kernel={kernel}({kernel_scale_km}km) | "
            f"sigma_total=RAW(SM) | imt={imtU}"
        )

    # ------------------------------------------------------------
    # MAIN LOOP over IMTs
    # ------------------------------------------------------------
    for imt in (IMT_LIST or []):
        imtU = str(imt).upper().strip()
        print("\n" + "=" * 70)
        print("[EXAMPLE C] GLOBAL EXTENT — IMT:", imtU)

        # -----------------------------
        # 1) Fresh instance (clean run)
        # -----------------------------
        shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
        shake.stations_folder = stations_folder
        shake.rupture_folder  = rupture_folder

        # -----------------------------
        # 2) Build dataset (ALWAYS under ./export/SHAKEuq/<event_id>/uq)
        # -----------------------------
        base_folder = UQ_BASE_FOLDER
        Path(base_folder).mkdir(parents=True, exist_ok=True)

        shake.uq_build_dataset(
            event_id=event_id,
            version_list=version_list_uq,
            base_folder=base_folder,                 # <-- FIXED policy
            stations_folder=stations_folder,
            rupture_folder=rupture_folder,
            imts=(imtU,),
            grid_unify="intersection",
            resolution="finest",
            export=True,
            interp_method="nearest",
            interp_kwargs=None,
        )
        print("[OK] uq_state built for", imtU)

        # -----------------------------
        # 3) Get unified axes consistent with chosen versions + IMT
        # -----------------------------
        _, lat2d, lon2d = shake._uq_get_unified_for_versions(
            version_list_uq,
            imt=imtU,
            grid_res=None,
            interp_method="nearest",
            interp_kwargs=None,
        )

        # -----------------------------
        # 4) Define GLOBAL extent as a bbox target (whole map)
        # -----------------------------
        global_bbox = [{
            "id": "GLOBAL_extent",
            "type": "area",
            "kind": "bbox",
            "minlat": float(np.nanmin(lat2d)),
            "maxlat": float(np.nanmax(lat2d)),
            "minlon": float(np.nanmin(lon2d)),
            "maxlon": float(np.nanmax(lon2d)),
        }]

        # Parse target and build mask
        t_global = shake._uq_parse_targets(points=None, areas=global_bbox)[0]
        mask_global, _meta = shake._uq_target_mask(t_global, lat2d, lon2d)

        # -----------------------------
        # 5) Published RAW min/max band per version over GLOBAL extent
        # -----------------------------
        xs, lo, hi = [], [], []
        for v in version_list_uq:
            band = shake._uq_band_minmax_unified(
                version=int(v),
                imt=imtU,
                lat2d=lat2d,
                lon2d=lon2d,
                mask=mask_global,
                grid_res=None,
                interp_method="nearest",
                interp_kwargs=None,
            )
            xs.append(int(v))
            lo.append(float(band["sig_min"]))
            hi.append(float(band["sig_max"]))

        # -----------------------------
        # 6) Extract GLOBAL mean series for all methods
        # -----------------------------
        df = shake.uq_extract_target_series(
            version_list=version_list_uq,
            imt=imtU,
            points=None,
            areas=global_bbox,
            agg="mean",
            global_stat=None,

            shakemap_total_sigma_mode="raw",
            sigma_total_from_shakemap=bool(WHICH_SIGMA_TOTAL),
            sigma_aleatory=sigma_aleatory_for_imt(imtU),
            prior_version=v_first,

            update_radius_km=update_radius_km,
            kernel=kernel,
            kernel_scale_km=kernel_scale_km,
            measurement_sigma=measurement_sigma_for_imt(imtU),

            ok_range_km=ok_range_km,
            ok_variogram=ok_variogram,
            ok_nugget=ok_nugget,
            ok_sill=ok_sill,
            ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

            mc_nsim=mc_nsim,
            mc_include_aleatory=mc_include_aleatory,

            grid_res=None,
            interp_method="nearest",
            interp_kwargs=None,

            audit=True,
            audit_output_path=OUT_ROOT,            # <-- FIXED policy
            audit_prefix=f"UQ-GlobalMean-{imtU}",
        )

        # -----------------------------
        # 7) Plot: published RAW band + method mean curves
        # -----------------------------
        fig, ax = plt.subplots(figsize=combined_figsize, dpi=dpi)

        ax.fill_between(
            xs, lo, hi,
            alpha=0.18,
            label="ShakeMap published RAW min/max (global extent)"
        )

        s_pub = df[df["method"] == "ShakeMap"].sort_values("version")
        if "sigma_total_published_raw" in s_pub.columns:
            ax.plot(
                s_pub["version"].astype(int).values,
                s_pub["sigma_total_published_raw"].astype(float).values,
                linewidth=linewidth,
                marker=marker_style,
                markersize=markersize,
                label="ShakeMap published RAW mean",
            )
        else:
            print("[WARN] DF missing column 'sigma_total_published_raw' for ShakeMap.")

        for m in ("bayes", "hierarchical", "kriging", "montecarlo"):
            s = df[df["method"] == m].sort_values("version")
            if len(s) == 0:
                continue
            if "sigma_total_predicted" not in s.columns:
                print(f"[WARN] DF missing 'sigma_total_predicted' for method={m}")
                continue
            ax.plot(
                s["version"].astype(int).values,
                s["sigma_total_predicted"].astype(float).values,
                linewidth=linewidth,
                marker=marker_style,
                markersize=markersize,
                label=f"{m} mean (predicted)",
            )

        if ylog:
            ax.set_yscale("log")
        if ymin is not None or ymax is not None:
            ax.set_ylim(ymin, ymax)

        ax.set_xlabel("ShakeMap version", fontsize=label_size)
        ax.set_ylabel(f"σ_total ({imtU})", fontsize=label_size)
        ax.tick_params(axis="both", labelsize=tick_size)
        ax.grid(True, alpha=0.35)

        ax.set_title(
            f"{imtU} GLOBAL decay (whole map extent): RAW min/max band + method mean curves\n{make_footer(imtU)}",
            fontsize=title_size,
        )

        lk = dict(legend_kwargs or {})
        lk.setdefault("fontsize", legend_size)
        ax.legend(**lk)

        fig.tight_layout()

        shake._uq_save_figure_safe(
            fig,
            fname_stem=f"UQ-ExampleC-GlobalBandPlusMeans-{imtU}",
            subdir="uq_plots/global_decay_",
            output_path=OUT_ROOT,                # <-- FIXED policy
            save_formats=SAVE_FORMATSs,
            dpi=dpi,
        )

        plt.show()
        print("[OK] Example C complete for", imtU)


In [ ]:
print('Section Cleared!')

## UQ DECAY SUITE

In [ ]:
%%time

figsize = (24, 15)

plt.style.use('./bins/latex_font.mplstyle')

# ============================================================
# EXAMPLE D: ONE-SHOT DECAY SUITE (POINT + AREA + GLOBAL)
# ============================================================
# One switch controls what is plotted everywhere.
# Runs:
#   A) POINT targets (points_city)
#   B) AREA targets (circles built from points_city)
#   C) GLOBAL extent (whole map bbox) + published RAW min/max band (sigma plots)
#
# NOTE:
# - AREA schema MUST be: {"type":"area","kind":"circle",...}
# - This example assumes your uq_plot_targets_decay signature uses:
#     markerstyle=..., markersize=...
# ============================================================

if run_decay_suite:

    import numpy as np
    import matplotlib.pyplot as plt

    # -----------------------------
    # USER SWITCH: what to plot
    # -----------------------------
    what = "sigma_epistemic_predicted"
    # Options you will likely use most:
    #   "sigma_total_predicted"      -> σ_total from methods (bayes/hier/kriging/mc)
    #   "sigma_epistemic_predicted"  -> σ_epistemic from methods
    #   "sigma_total_published_raw"  -> published ShakeMap RAW σ_total baseline
    #   "mean_predicted"             -> predicted mean (IMT)
    #   "mean_published"             -> published mean (IMT)
    #   "delta_sigma_vs_published"   -> predicted σ_total - published RAW σ_total
    #   "delta_mean_vs_published"    -> predicted mean - published mean

    methods = ("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo")
    versions = sorted([int(v) for v in version_list_uq])

    # -----------------------------
    # Build AREA circles from points_city
    # -----------------------------
    radius_km = 30.0
    areas_from_points = [
        {
            "id": f"{p['id']}_R{int(radius_km)}km",
            "type": "area",
            "kind": "circle",
            "lat": float(p["lat"]),
            "lon": float(p["lon"]),
            "radius_km": float(radius_km),
        }
        for p in points_city
    ]

    # ============================================================
    # A) POINT DECAY
    # ============================================================
    df_point = shake.uq_plot_targets_decay(
        version_list=versions, imt=imtU,
        points=points_city, areas=None,
        what=what, methods=methods, agg="mean", global_stat=None, prior_version=v_first,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km, kernel=kernel, kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km, ok_variogram=ok_variogram, ok_nugget=ok_nugget,
        ok_sill=ok_sill, ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim, mc_include_aleatory=mc_include_aleatory,

        grid_res=None, interp_method="nearest", interp_kwargs=None,

        figsize=figsize, dpi=dpi, ylog=ylog, ymin=ymin, ymax=ymax, xrotation=xrotation,
        show_title=True,
        title=f"{imtU} decay @ {{target}} (POINT) — {what}\n{footer}",
        show_grid=True, legend=True, legend_kwargs=legend_kwargs, tight=True,

        label_size=label_size, tick_size=tick_size, title_size=title_size, legend_size=legend_size,
        xlabel="version", ylabel=f"{what} ({imtU})",

        linewidth=linewidth, markerstyle=marker_style, markersize=markersize,

        output_path=out_root, save=True, save_formats=SAVE_FORMATSs, show=True,

        plot_combined=True, combined_figsize=combined_figsize, combined_legend_ncol=2,

        audit=True, audit_output_path=out_root, audit_prefix=f"UQ-SUITE-POINT-{imtU}-{what}",
    )
    print("[OK] POINT complete")

    # ============================================================
    # B) AREA DECAY (circle around each city)
    # ============================================================
    df_area = shake.uq_plot_targets_decay(
        version_list=versions, imt=imtU,
        points=None, areas=areas_from_points,
        what=what, methods=methods, agg="mean", global_stat=None, prior_version=v_first,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km, kernel=kernel, kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km, ok_variogram=ok_variogram, ok_nugget=ok_nugget,
        ok_sill=ok_sill, ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim, mc_include_aleatory=mc_include_aleatory,

        grid_res=None, interp_method="nearest", interp_kwargs=None,

        figsize=figsize, dpi=dpi, ylog=ylog, ymin=ymin, ymax=ymax, xrotation=xrotation,
        show_title=True,
        title=f"{imtU} decay @ {{target}} (AREA circle, R={radius_km:.0f} km) — {what}\n{footer}",
        show_grid=True, legend=True, legend_kwargs=legend_kwargs, tight=True,

        label_size=label_size, tick_size=tick_size, title_size=title_size, legend_size=legend_size,
        xlabel="version", ylabel=f"{what} ({imtU})",

        linewidth=linewidth, markerstyle=marker_style, markersize=markersize,

        output_path=out_root, save=True, save_formats=SAVE_FORMATSs, show=True,

        plot_combined=True, combined_figsize=combined_figsize, combined_legend_ncol=2,

        # Band shading: only matters for AREA/GLOBAL + ShakeMap present
        published_band=True, published_band_alpha=0.18,

        audit=True, audit_output_path=out_root, audit_prefix=f"UQ-SUITE-AREA-{imtU}-{what}",
    )
    print("[OK] AREA complete")

    # ============================================================
    # C) GLOBAL EXTENT (whole map) + published RAW min/max band (sigma plots)
    # ============================================================
    # unified axes for these versions/imt
    _, lat2d, lon2d = shake._uq_get_unified_for_versions(
        versions, imt=str(imtU).upper(),
        grid_res=None, interp_method="nearest", interp_kwargs=None
    )

    global_bbox = [{
        "id": "GLOBAL_extent",
        "type": "area",
        "kind": "bbox",
        "minlat": float(np.nanmin(lat2d)),
        "maxlat": float(np.nanmax(lat2d)),
        "minlon": float(np.nanmin(lon2d)),
        "maxlon": float(np.nanmax(lon2d)),
    }]

    # GLOBAL mean series (all methods)
    df_global = shake.uq_extract_target_series(
        version_list=versions, imt=imtU,
        points=None, areas=global_bbox,
        agg="mean", global_stat=None,
        shakemap_total_sigma_mode="raw",

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),
        prior_version=v_first,

        update_radius_km=update_radius_km, kernel=kernel, kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km, ok_variogram=ok_variogram, ok_nugget=ok_nugget,
        ok_sill=ok_sill, ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim, mc_include_aleatory=mc_include_aleatory,

        grid_res=None, interp_method="nearest", interp_kwargs=None,

        audit=True, audit_output_path=out_root, audit_prefix=f"UQ-SUITE-GLOBAL-{imtU}-{what}",
    )

    # Published RAW min/max band (only meaningful for sigma-like plots)
    wL = str(what).lower()
    do_band = ("sigma" in wL) and ("delta" not in wL)

    fig, ax = plt.subplots(figsize=combined_figsize, dpi=dpi)

    if do_band:
        # mask for bbox (whole map)
        t = shake._uq_parse_targets(points=None, areas=global_bbox)[0]
        mask, _ = shake._uq_target_mask(t, lat2d, lon2d)

        xs, lo, hi = [], [], []
        for v in versions:
            band = shake._uq_band_minmax_unified(
                version=int(v), imt=str(imtU).upper(),
                lat2d=lat2d, lon2d=lon2d, mask=mask,
                grid_res=None, interp_method="nearest", interp_kwargs=None
            )
            xs.append(int(v))
            lo.append(float(band["sig_min"]))
            hi.append(float(band["sig_max"]))
        ax.fill_between(xs, lo, hi, alpha=0.18, label="ShakeMap RAW min/max (global extent)")

    # plot global mean curves
    for m in methods:
        s = df_global[df_global["method"] == m].sort_values("version")
        if len(s) == 0:
            continue

        # select appropriate column per method (published vs predicted)
        col = what
        if m == "ShakeMap":
            if what == "sigma_total_predicted":
                col = "sigma_total_published_raw"
            if what == "mean_predicted":
                col = "mean_published"
            if what == "sigma_epistemic_predicted":
                col = "sigma_total_published_raw"  # no published epistemic; show RAW total baseline
        else:
            if what == "sigma_total_published_raw":
                col = "sigma_total_predicted"
            if what == "mean_published":
                col = "mean_predicted"

        if col not in s.columns:
            continue

        ax.plot(
            s["version"].astype(int).values,
            s[col].astype(float).values,
            linewidth=linewidth,
            marker=marker_style,
            markersize=markersize,
            label=m,
        )

    if ylog:
        ax.set_yscale("log")
    if ymin is not None or ymax is not None:
        ax.set_ylim(ymin, ymax)

    ax.set_xlabel("ShakeMap version", fontsize=label_size)
    ax.set_ylabel(f"{what} ({imtU})", fontsize=label_size)
    ax.tick_params(axis="both", labelsize=tick_size)
    ax.grid(True, alpha=0.35)
    ax.set_title(f"{imtU} GLOBAL extent decay — {what}\n{footer}", fontsize=title_size)

    lk = dict(legend_kwargs or {})
    lk.setdefault("fontsize", legend_size)
    ax.legend(**lk)
    fig.tight_layout()

    shake._uq_save_figure_safe(
        fig,
        fname_stem=f"UQ-SUITE-GLOBAL-{imtU}-{what}",
        subdir="uq_plots/decay_suite_global",
        output_path=out_root,
        save_formats=SAVE_FORMATSs,
        dpi=dpi,
    )
    plt.show()
    print("[OK] GLOBAL complete")

    print("\n[DONE] Decay suite finished.")
    display(df_point.head(5))
    display(df_area.head(5))
    display(df_global.head(5))


In [ ]:
print('Section Cleared!')

# Target Predictions Decay 

This section focuses on the **predicted mean field evolution** (e.g., MMI or PGA) at targets across versions. It complements the uncertainty decay section by showing:
- whether predicted means stabilize over time,
- how updates shift target estimates,
- and how method-based predictions compare to the published baseline trend.


In [ ]:
%%time

plt.style.use('./bins/latex_font.mplstyle')

# ============================================================
# EXAMPLE E (Standalone, Modular): ONE-SHOT DECAY SUITE
#   - Builds its own SHAKEuq instance
#   - Builds its own unified dataset (saved to: <OUT_ROOT>/SHAKEuq/<event_id>/uq)
#   - Loops over IMTs you request
#
# What this example produces (for each IMT):
#   A) POINT target decay curves  (points_city)
#   B) AREA  target decay curves  (circles built from points_city)
#   C) GLOBAL extent curves       (whole-map bbox) + optional RAW min/max band (sigma only)
#
# Primary purpose (your note):
#   Plot TARGET MEAN decay predictions (default: what="mean_predicted"),
#   but you can switch "what" to sigma / deltas and it will still run.
#
# REQUIREMENTS from your event setup cell:
#   event_id, event_time, shakemap_folder, pager_folder
#   stations_folder, rupture_folder, version_list (or version_list_uq)
#   points_city  (list of dicts with at least id/lat/lon)
# ============================================================

if run_mean_suite:

    from modules.SHAKEtime import SHAKEtime
    from pathlib import Path
    import numpy as np
    import matplotlib.pyplot as plt

    # ------------------------------------------------------------
    # USER KNOBS (change only these)
    # ------------------------------------------------------------
    IMT_LIST = run_mean_suite    # ["MMI"] or ["PGA"] or []/None to skip
    OUT_ROOT = "./export"        # base root; dataset will go to: OUT_ROOT/SHAKEuq/<event_id>/uq
    SAVE_FORMATS = SAVE_FORMATSs
    DPI = FIGDPIs

    # What to plot everywhere
    what = "mean_predicted"
    # Common options:
    #   "mean_predicted"
    #   "mean_published"
    #   "sigma_total_predicted"
    #   "sigma_total_published_raw"
    #   "delta_mean_vs_published"
    #   "delta_sigma_vs_published"

    METHODS = ("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo")

    # Area definition (for AREA curves)
    radius_km = 30.0

    # Unified grid knobs (match your other examples)
    grid_res = None
    interp_method = "nearest"
    interp_kwargs = None

    # ------------------------------------------------------------
    # UQ MODEL KNOBS (keep aligned with Examples A/B/C)
    # ------------------------------------------------------------
    sigma_total_from_shakemap = True  # RAW ShakeMap σ_total is baseline

    SIGMA_ALEATORY_MMI = 0.40
    SIGMA_ALEATORY_PGA = 0.35

    def sigma_aleatory_for_imt(imtU):
        imtU0 = str(imtU).upper().strip()
        if imtU0 == "MMI":
            return SIGMA_ALEATORY_MMI
        if imtU0 in ("PGA", "PGV"):
            return SIGMA_ALEATORY_PGA
        return None

    update_radius_km = 25.0
    kernel = "gaussian"
    kernel_scale_km = 20.0

    MEAS_SIGMA_MMI = 0.30
    MEAS_SIGMA_PGA = 0.35

    def measurement_sigma_for_imt(imtU):
        imtU0 = str(imtU).upper().strip()
        if imtU0 == "MMI":
            return MEAS_SIGMA_MMI
        if imtU0 == "PGA":
            return MEAS_SIGMA_PGA
        return 0.30

    ok_range_km = 60.0
    ok_variogram = "exponential"
    ok_nugget = 1e-6
    ok_sill = None
    ok_cap_sigma_to_prior = True

    mc_nsim = 500
    mc_include_aleatory = True

    # ------------------------------------------------------------
    # PLOT STYLE (same pattern as A/B)
    # ------------------------------------------------------------
    FIGSIZE = FIGSIZE_data_plot
    COMBINED_FIGSIZE = FIGSIZE_full_page
    ylog = False
    ymin = None
    ymax = None
    xrotation = 0

    label_size  = LABEL_SIZEs
    tick_size   = TICKS_SIZEs
    title_size  = TITLE_SIZEs
    legend_size = LEGEND_SIZEs

    linewidth = LINEWIDTHs
    marker_style = MARKER_STYLEs
    markersize = MARKER_SIZEs

    legend_kwargs = LEGEND_KWARGs

    # Optional style:
    # plt.style.use("./bins/latex_font.mplstyle")

    # ------------------------------------------------------------
    # REQUIRED from your event setup cell
    # ------------------------------------------------------------
    versions = sorted([int(v) for v in (globals().get("version_list_uq", None) or version_list)])
    v_first = int(versions[0])

    # Ensure consistent output root exists
    Path(OUT_ROOT).mkdir(parents=True, exist_ok=True)

    # ------------------------------------------------------------
    # Helper: footer
    # ------------------------------------------------------------
    def make_footer(imtU):
        return (
            f"event={event_id} | prior=v{v_first:03d} | "
            f"updateR={update_radius_km:.0f}km | kernel={kernel}({kernel_scale_km:.0f}km) | "
            f"sigma_total=RAW(SM) | imt={imtU}"
        )

    # ------------------------------------------------------------
    # MAIN LOOP over IMTs
    # ------------------------------------------------------------
    IMT_LIST = IMT_LIST or []
    for imt in IMT_LIST:
        imtU = str(imt).upper().strip()

        print("\n" + "=" * 80)
        print(f"[EXAMPLE E] ONE-SHOT DECAY SUITE | IMT={imtU} | what={what}")
        print("  OUT_ROOT:", OUT_ROOT)
        print("  versions:", versions[0], "→", versions[-1], "| n =", len(versions))

        # ------------------------------------------------------------
        # 0) Fresh SHAKEuq instance
        # ------------------------------------------------------------
        shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
        shake.stations_folder = stations_folder
        shake.rupture_folder  = rupture_folder

        # ------------------------------------------------------------
        # 1) Build unified dataset
        #    IMPORTANT: base_folder should be OUT_ROOT (NOT OUT_ROOT/SHAKEuq)
        #    because uq_build_dataset now enforces:
        #      OUT_ROOT/SHAKEuq/<event_id>/uq
        # ------------------------------------------------------------
        shake.uq_build_dataset(
            event_id=event_id,
            version_list=versions,
            base_folder=OUT_ROOT,
            stations_folder=stations_folder,
            rupture_folder=rupture_folder,
            imts=(imtU,),
            grid_unify="intersection",
            resolution="finest",
            export=True,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,
        )
        print("[OK] uq_state built | base_folder =", shake.uq_state.get("base_folder"))

        # ------------------------------------------------------------
        # 2) Targets: POINTS schema + AREAS schema
        # ------------------------------------------------------------
        points_uq = []
        for p in points_city:
            pp = dict(p)
            pp.setdefault("type", "point")
            pp["id"]  = str(pp.get("id", pp.get("name", "point")))
            pp["lat"] = float(pp["lat"])
            pp["lon"] = float(pp["lon"])
            points_uq.append(pp)

        areas_uq = []
        for p in points_uq:
            areas_uq.append(
                {
                    "id": f"{p['id']}_R{int(radius_km)}km",
                    "type": "area",
                    "kind": "circle",
                    "lat": float(p["lat"]),
                    "lon": float(p["lon"]),
                    "radius_km": float(radius_km),
                }
            )

        footer = make_footer(imtU)

        # ============================================================
        # A) POINT DECAY
        # ============================================================
        df_point = shake.uq_plot_targets_decay(
            version_list=versions,
            imt=imtU,

            points=points_uq,
            areas=None,

            what=what,
            methods=METHODS,
            agg="mean",
            global_stat=None,
            prior_version=v_first,

            sigma_total_from_shakemap=sigma_total_from_shakemap,
            sigma_aleatory=sigma_aleatory_for_imt(imtU),

            update_radius_km=update_radius_km,
            kernel=kernel,
            kernel_scale_km=kernel_scale_km,
            measurement_sigma=measurement_sigma_for_imt(imtU),

            ok_range_km=ok_range_km,
            ok_variogram=ok_variogram,
            ok_nugget=ok_nugget,
            ok_sill=ok_sill,
            ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

            mc_nsim=mc_nsim,
            mc_include_aleatory=mc_include_aleatory,

            grid_res=grid_res,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,

            figsize=FIGSIZE,
            dpi=DPI,
            ylog=ylog,
            ymin=ymin,
            ymax=ymax,
            xrotation=xrotation,

            show_title=True,
            title=f"{imtU} decay @ {{target}} (POINT) — {what}\n{footer}",
            show_grid=True,
            legend=True,
            legend_kwargs=legend_kwargs,
            tight=True,

            label_size=label_size,
            tick_size=tick_size,
            title_size=title_size,
            legend_size=legend_size,
            xlabel="version",
            ylabel=f"{what} ({imtU})",

            linewidth=linewidth,
            markerstyle=marker_style,
            markersize=markersize,

            output_path=OUT_ROOT,
            save=True,
            save_formats=SAVE_FORMATS,
            show=True,

            plot_combined=True,
            combined_figsize=COMBINED_FIGSIZE,
            combined_legend_ncol=2,

            audit=True,
            audit_output_path=OUT_ROOT,
            audit_prefix=f"UQ-SUITE-POINT-{imtU}-{what}",
        )
        print("[OK] POINT complete")

        # ============================================================
        # B) AREA DECAY (circle around each city)
        # ============================================================
        df_area = shake.uq_plot_targets_decay(
            version_list=versions,
            imt=imtU,

            points=None,
            areas=areas_uq,

            what=what,
            methods=METHODS,
            agg="mean",
            global_stat=None,
            prior_version=v_first,

            sigma_total_from_shakemap=sigma_total_from_shakemap,
            sigma_aleatory=sigma_aleatory_for_imt(imtU),

            update_radius_km=update_radius_km,
            kernel=kernel,
            kernel_scale_km=kernel_scale_km,
            measurement_sigma=measurement_sigma_for_imt(imtU),

            ok_range_km=ok_range_km,
            ok_variogram=ok_variogram,
            ok_nugget=ok_nugget,
            ok_sill=ok_sill,
            ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

            mc_nsim=mc_nsim,
            mc_include_aleatory=mc_include_aleatory,

            grid_res=grid_res,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,

            figsize=FIGSIZE,
            dpi=DPI,
            ylog=ylog,
            ymin=ymin,
            ymax=ymax,
            xrotation=xrotation,

            show_title=True,
            title=f"{imtU} decay @ {{target}} (AREA circle, R={radius_km:.0f} km) — {what}\n{footer}",
            show_grid=True,
            legend=True,
            legend_kwargs=legend_kwargs,
            tight=True,

            label_size=label_size,
            tick_size=tick_size,
            title_size=title_size,
            legend_size=legend_size,
            xlabel="version",
            ylabel=f"{what} ({imtU})",

            linewidth=linewidth,
            markerstyle=marker_style,
            markersize=markersize,

            output_path=OUT_ROOT,
            save=True,
            save_formats=SAVE_FORMATS,
            show=True,

            plot_combined=True,
            combined_figsize=COMBINED_FIGSIZE,
            combined_legend_ncol=2,

            # Band shading (only meaningful when ShakeMap + sigma-style plots)
            published_band=True,
            published_band_alpha=0.18,

            audit=True,
            audit_output_path=OUT_ROOT,
            audit_prefix=f"UQ-SUITE-AREA-{imtU}-{what}",
        )
        print("[OK] AREA complete")

        # ============================================================
        # C) GLOBAL EXTENT (whole map) + optional RAW min/max band (sigma only)
        # ============================================================
        _, lat2d, lon2d = shake._uq_get_unified_for_versions(
            versions,
            imt=imtU,
            grid_res=grid_res,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,
        )

        global_bbox = [
            {
                "id": "GLOBAL_extent",
                "type": "area",
                "kind": "bbox",
                "minlat": float(np.nanmin(lat2d)),
                "maxlat": float(np.nanmax(lat2d)),
                "minlon": float(np.nanmin(lon2d)),
                "maxlon": float(np.nanmax(lon2d)),
            }
        ]

        df_global = shake.uq_extract_target_series(
            version_list=versions,
            imt=imtU,

            points=None,
            areas=global_bbox,

            agg="mean",
            global_stat=None,

            shakemap_total_sigma_mode="raw",
            sigma_total_from_shakemap=sigma_total_from_shakemap,
            sigma_aleatory=sigma_aleatory_for_imt(imtU),
            prior_version=v_first,

            update_radius_km=update_radius_km,
            kernel=kernel,
            kernel_scale_km=kernel_scale_km,
            measurement_sigma=measurement_sigma_for_imt(imtU),

            ok_range_km=ok_range_km,
            ok_variogram=ok_variogram,
            ok_nugget=ok_nugget,
            ok_sill=ok_sill,
            ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

            mc_nsim=mc_nsim,
            mc_include_aleatory=mc_include_aleatory,

            grid_res=grid_res,
            interp_method=interp_method,
            interp_kwargs=interp_kwargs,

            audit=True,
            audit_output_path=OUT_ROOT,
            audit_prefix=f"UQ-SUITE-GLOBAL-{imtU}-{what}",
        )

        # Sigma band only when "sigma" and not "delta"
        wL = str(what).lower()
        do_band = ("sigma" in wL) and ("delta" not in wL)

        fig, ax = plt.subplots(figsize=COMBINED_FIGSIZE, dpi=DPI)

        if do_band:
            t = shake._uq_parse_targets(points=None, areas=global_bbox)[0]
            mask, _ = shake._uq_target_mask(t, lat2d, lon2d)

            xs, lo, hi = [], [], []
            for v in versions:
                band = shake._uq_band_minmax_unified(
                    version=int(v),
                    imt=imtU,
                    lat2d=lat2d,
                    lon2d=lon2d,
                    mask=mask,
                    grid_res=grid_res,
                    interp_method=interp_method,
                    interp_kwargs=interp_kwargs,
                )
                xs.append(int(v))
                lo.append(float(band["sig_min"]))
                hi.append(float(band["sig_max"]))
            ax.fill_between(xs, lo, hi, alpha=0.18, label="ShakeMap RAW min/max (global extent)")

        # Plot method curves (choose appropriate column mappings)
        for m in METHODS:
            s = df_global[df_global["method"] == m].sort_values("version")
            if len(s) == 0:
                continue

            col = what
            if m == "ShakeMap":
                if what == "sigma_total_predicted":
                    col = "sigma_total_published_raw"
                elif what == "mean_predicted":
                    col = "mean_published"
                elif what == "sigma_epistemic_predicted":
                    col = "sigma_total_published_raw"
            else:
                if what == "sigma_total_published_raw":
                    col = "sigma_total_predicted"
                elif what == "mean_published":
                    col = "mean_predicted"

            if col not in s.columns:
                continue

            ax.plot(
                s["version"].astype(int).values,
                s[col].astype(float).values,
                linewidth=linewidth,
                marker=marker_style,
                markersize=markersize,
                label=m,
            )

        if ylog:
            ax.set_yscale("log")
        if ymin is not None or ymax is not None:
            ax.set_ylim(ymin, ymax)

        ax.set_xlabel("ShakeMap version", fontsize=label_size)
        ax.set_ylabel(f"{what} ({imtU})", fontsize=label_size)
        ax.tick_params(axis="both", labelsize=tick_size)
        ax.grid(True, alpha=0.35)
        ax.set_title(f"{imtU} GLOBAL extent decay — {what}\n{footer}", fontsize=title_size)

        lk = dict(legend_kwargs or {})
        lk.setdefault("fontsize", legend_size)
        ax.legend(**lk)
        fig.tight_layout()

        shake._uq_save_figure_safe(
            fig,
            fname_stem=f"UQ-SUITE-GLOBAL-{imtU}-{what}",
            subdir="uq_plots/target_mean_decay",
            output_path=OUT_ROOT,
            save_formats=SAVE_FORMATS,
            dpi=DPI,
        )

        plt.show()
        print("[OK] GLOBAL complete")

        print("\n[DONE] Decay suite finished for IMT =", imtU)
        display(df_point.head(5))
        display(df_area.head(5))
        display(df_global.head(5))


In [ ]:
print('Section Cleared!')

# Sigme Reduction Map 

This section produces **map-based diagnostics** of uncertainty:
- prior (published) sigma maps,
- posterior sigma maps (method-dependent),
- change maps (first vs last, or sequential updates),
- and reduction maps (prior − post).

These plots answer: **where** uncertainty decreases, **how much**, and **whether the spatial pattern matches expectations** given new observations and updates.


In [ ]:
%%time
plt.style.use('./bins/latex_font.mplstyle')


if plot_sigma_maps:

    from modules.SHAKEtime import SHAKEtime
    from pathlib import Path

    # ------------------------------------------------------------------
    # IMPORTANT PATH POLICY (FIX)
    #   - uq_build_dataset(base_folder=...) should point to the folder that
    #     will contain: <base_folder>/<event_id>/uq/...
    #     so we set it to "./export/SHAKEuq"
    #
    #   - uq_plot_uq_sigma_map(output_path=...) internally appends:
    #       /SHAKEuq/<event_id>/uq/...
    #     so output_path must be "./export" (NOT "./export/SHAKEuq")
    #
    # This guarantees everything lands under:
    #   ./export/SHAKEuq/<event_id>/uq/...
    # ------------------------------------------------------------------
    DATASET_BASE = "./export"   # dataset writes here: ./export/SHAKEuq/<event_id>/uq/...
    EXPORT_ROOT  = "./export"            # maps/posteriors write here (function appends /SHAKEuq/<event_id>/uq/...)

    #imt_list = [ "MMI" , "PGA" ]

    for imts in plot_sigma_maps:
        # ============================================================
        # PATCH 4.2 — REQUIRED MAP SET (ShakeMap RAW + Bayes + Hier)
        #   (v26.5 plotting knobs pre-defined + reused)
        # ============================================================

        # ------------------------------------------------------------
        # USER KNOBS (change only these)
        # ------------------------------------------------------------
        IMT = imts                # "MMI" or "PGA"
        WHICH_SIGMA = "total"     # "total" or "epistemic"
        OUT_ROOT = EXPORT_ROOT    # <-- FIX: plotting root must be "./export"

        # Bayes knobs (keep simple)
        BAYES_update_radius_km = 25.0
        BAYES_update_kernel = "gaussian"
        BAYES_measurement_sigma = 0.30 if IMT.upper() == "MMI" else 0.35
        BAYES_sigma_aleatory = 0.40 if IMT.upper() == "MMI" else 0.35
        BAYES_sigma_total_from_shakemap = True
        BAYES_make_audit = False

        # Hierarchical knobs (simple defaults)
        HIER_update_radius_km = 30.0
        HIER_kernel = "gaussian"
        HIER_kernel_scale_km = 20.0
        HIER_measurement_sigma = 0.30 if IMT.upper() == "MMI" else 0.35

        # Plot styling
        FIGSIZE = FIGSIZE_paired_maps
        DPI = FIGDPIs
        SAVE_FORMATS = SAVE_FORMATSs

        # ------------------------------------------------------------
        # NEW: Shared plotting kwargs (font + layout + colorbar)
        # ------------------------------------------------------------

        # Font sizes
        TITLE_FONTSIZE = TITLE_SIZEs          # suptitle (panel) / title (single)
        SUBTITLE_FONTSIZE = TITLE_SIZEs      # each panel title (v001, v002, etc.)
        CBAR_LABEL_FONTSIZE = LABEL_SIZEs
        CBAR_TICK_FONTSIZE = LABEL_SIZEs

        # Panel spacing (use if panels overlap)
        PANEL_WSPACE = 0.08
        PANEL_HSPACE = 0.2
        PANEL_RIGHT = 0.86           # reserve space for outside colorbar
        PANEL_LEFT  = 0.06
        PANEL_TOP   = 0.92
        PANEL_BOTTOM= 0.06

        # Outside colorbar placement (figure fraction) — easy to tune
        CBAR_OUTSIDE = True
        CBAR_RECT_DEFAULT = (0.88, 0.15, 0.025, 0.70)   # (x0,y0,w,h)

        # Common kwargs for ALL maps
        PLOT_KW = dict(
            figsize=FIGSIZE,
            dpi=DPI,
            save=True,
            save_formats=SAVE_FORMATS,
            show=True,

            # fonts
            title_fontsize=TITLE_FONTSIZE,
            subplot_title_fontsize=SUBTITLE_FONTSIZE,
            cbar_label_fontsize=CBAR_LABEL_FONTSIZE,
            cbar_tick_fontsize=CBAR_TICK_FONTSIZE,

            # colorbar + spacing (panel mode uses these; single mode ignores most)
            cbar_outside=CBAR_OUTSIDE,
            cbar_rect=CBAR_RECT_DEFAULT,
            panel_wspace=PANEL_WSPACE,
            panel_hspace=PANEL_HSPACE,
            panel_right=PANEL_RIGHT,
            panel_left=PANEL_LEFT,
            panel_top=PANEL_TOP,
            panel_bottom=PANEL_BOTTOM,
            panel_use_tight_layout=False,  # let subplots_adjust control it (cartopy-friendly)
        )

        # ------------------------------------------------------------
        # NEW: Change-map colormap (diverging, centered at 0)
        #   - 0 ~ white, negative ~ red, positive ~ blue
        #   - IMPORTANT: to FORCE 0 centered, set CHANGE_ABS_MAX (symmetric vmin/vmax)
        # ------------------------------------------------------------
        CHANGE_CMAP = "seismic"
        CHANGE_ABS_MAX = 0.25   # <-- set this to force 0-centered scaling; or set None for auto

        def CHANGE_KW():
            if CHANGE_ABS_MAX is not None:
                V = abs(float(CHANGE_ABS_MAX))
                return dict(cmap=CHANGE_CMAP, vmin=-V, vmax=+V)
            return dict(cmap=CHANGE_CMAP)  # auto-range (may NOT be perfectly 0-centered)

        # ------------------------------------------------------------
        # REQUIRED from your event setup cell
        # ------------------------------------------------------------
        # event_id, event_time, shakemap_folder, pager_folder
        # stations_folder, rupture_folder, version_list
        # ------------------------------------------------------------

        imtU = IMT.upper()
        vlist = sorted([int(v) for v in version_list])
        v_first, v_last = vlist[0], vlist[-1]

        Path(OUT_ROOT).mkdir(parents=True, exist_ok=True)

        # Make fresh instance (clean run)
        shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
        shake.stations_folder = stations_folder
        shake.rupture_folder  = rupture_folder

        print("[OK] event:", event_id, "| IMT:", imtU, "| sigma:", WHICH_SIGMA)
        print("[OK] versions:", v_first, "→", v_last, "| n =", len(vlist))

        # ------------------------------------------------------------
        # 0) Build unified dataset (needed for mapping axes + ShakeMap RAW fields)
        # ------------------------------------------------------------
        base_folder = DATASET_BASE   # <-- FIX: dataset base must be "./export/SHAKEuq"
        Path(base_folder).mkdir(parents=True, exist_ok=True)

        shake.uq_build_dataset(
            event_id=event_id,
            version_list=vlist,
            base_folder=base_folder,
            stations_folder=stations_folder,
            rupture_folder=rupture_folder,
            imts=(imtU,),
            grid_unify="intersection",
            resolution="finest",
            export=True,
            interp_method="nearest",
            interp_kwargs=None,
        )
        print("[OK] uq_state built")
        print("[DEBUG] uq_state base_folder:", shake.uq_state.get("base_folder"))

        # ============================================================
        # A) ShakeMap RAW total uncertainty maps (FIRST + LAST)
        # ============================================================

        print("\n[A1] ShakeMap RAW sigma — FIRST")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="shakemap",
            kind="prior",                 # ShakeMap published sigma
            which_sigma=WHICH_SIGMA,      # total or epistemic
            version=v_first,
            output_path=OUT_ROOT,
            **PLOT_KW,
        )

        print("\n[A2] ShakeMap RAW sigma — LAST")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="shakemap",
            kind="prior",
            which_sigma=WHICH_SIGMA,
            version=v_last,
            output_path=OUT_ROOT,
            **PLOT_KW,
        )

        # ============================================================
        # B) ShakeMap RAW change map (FIRST vs LAST) + panel of steps
        # ============================================================

        print("\n[B1] ShakeMap RAW CHANGE (FIRST vs LAST)")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="shakemap",
            kind="change",
            which_sigma=WHICH_SIGMA,
            first_version=v_first,
            last_version=v_last,
            output_path=OUT_ROOT,
            **PLOT_KW,
            **CHANGE_KW(),
        )

        print("\n[B2] ShakeMap RAW CHANGE PANELS (cumulative first->each, plus FIRST vs LAST)")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="shakemap",
            kind="change",
            which_sigma=WHICH_SIGMA,
            panel_all=True,
            panel_mode="cumulative",      # first -> each
            panel_ncol=3,
            include_last_vs_first_panel=True,
            output_path=OUT_ROOT,
            **PLOT_KW,
            **CHANGE_KW(),panel_figsize=FIGSIZE_full_page,obs_size= 5,
        )

        # ============================================================
        # C) Final version POST total uncertainty (BayesUpdate vs Hierarchical)
        # ============================================================

        print("\n[C1] BayesUpdate POST sigma — LAST")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="bayesupdate",
            kind="post",
            which_sigma=WHICH_SIGMA,
            version=v_last,

            compute_if_missing=True,
            export_posteriors=True,
            posterior_output_path=OUT_ROOT,   # <-- FIX: must match plot root ("./export")

            update_radius_km=BAYES_update_radius_km,
            update_kernel=BAYES_update_kernel,
            measurement_sigma=BAYES_measurement_sigma,
            sigma_total_from_shakemap=BAYES_sigma_total_from_shakemap,
            sigma_aleatory=BAYES_sigma_aleatory,
            make_audit=BAYES_make_audit,

            output_path=OUT_ROOT,
            **PLOT_KW,
        )

        print("\n[C2] Hierarchical POST sigma — LAST")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="hierarchical",
            kind="post",
            which_sigma=WHICH_SIGMA,
            version=v_last,

            compute_if_missing=True,
            export_posteriors=True,
            posterior_output_path=OUT_ROOT,   # <-- FIX: must match plot root ("./export")

            hier_update_radius_km=HIER_update_radius_km,
            hier_kernel=HIER_kernel,
            hier_kernel_scale_km=HIER_kernel_scale_km,
            hier_measurement_sigma=HIER_measurement_sigma,

            output_path=OUT_ROOT,
            **PLOT_KW,
        )

        # ============================================================
        # D) Change in uncertainty (FIRST vs LAST) for Bayes & Hier (POST change)
        # ============================================================

        print("\n[D1] BayesUpdate POST CHANGE (FIRST vs LAST)")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="bayesupdate",
            kind="change",
            which_sigma=WHICH_SIGMA,
            first_version=v_first,
            last_version=v_last,

            compute_if_missing=True,
            export_posteriors=True,
            posterior_output_path=OUT_ROOT,   # <-- FIX: must match plot root ("./export")

            update_radius_km=BAYES_update_radius_km,
            update_kernel=BAYES_update_kernel,
            measurement_sigma=BAYES_measurement_sigma,
            sigma_total_from_shakemap=BAYES_sigma_total_from_shakemap,
            sigma_aleatory=BAYES_sigma_aleatory,
            make_audit=BAYES_make_audit,

            output_path=OUT_ROOT,
            panel_all=True,
            panel_mode="cumulative",      # first -> each
            panel_ncol=3,
            **PLOT_KW,
            **CHANGE_KW(),panel_figsize=FIGSIZE_full_page,obs_size= 5,
        )

        print("\n[D2] Hierarchical POST CHANGE (FIRST vs LAST)")
        shake.uq_plot_uq_sigma_map(
            version_list=vlist,
            imt=imtU,
            method="hierarchical",
            kind="change",
            which_sigma=WHICH_SIGMA,
            first_version=v_first,
            last_version=v_last,

            compute_if_missing=True,
            export_posteriors=True,
            posterior_output_path=OUT_ROOT,   # <-- FIX: must match plot root ("./export")

            hier_update_radius_km=HIER_update_radius_km,
            hier_kernel=HIER_kernel,
            hier_kernel_scale_km=HIER_kernel_scale_km,
            hier_measurement_sigma=HIER_measurement_sigma,

            output_path=OUT_ROOT,
            panel_all=True,
            panel_mode="cumulative",      # first -> each
            panel_ncol=3,
            **PLOT_KW,
            **CHANGE_KW(),panel_figsize=FIGSIZE_full_page,obs_size= 5,
        )

        print("\n[OK] All requested maps exported under:")
        print(f"  {EXPORT_ROOT}/SHAKEuq/{event_id}/uq/uq_maps/")
        print(f"  {EXPORT_ROOT}/SHAKEuq/{event_id}/uq/uq_posteriors/   (if computed)")
        print(f"[OK] Dataset stored under:")
        print(f"  {DATASET_BASE}/{event_id}/uq/")


In [ ]:
print('Section Cleared!')

# RAW vs Unified Grid Validation

This section verifies that mapping version products onto the **unified grid** does not introduce unintended artifacts. You compare:
- values sampled directly from RAW ShakeMap XML (grid/uncertainty),
- versus values taken from the unified-grid representation,
for both points and areas.

The goal is to confirm that any differences in later analyses are **real workflow effects**, not grid-alignment or interpolation mistakes.


In [ ]:
%%time

plt.style.use('./bins/latex_font.mplstyle')

# ============================================================
# RAW vs UNIFIED GRID VALIDATION — Patch 4.1-G (standalone notebook run)
#   - Creates SHAKEuq instance (using current __init__ signature)
#   - Hotfixes unified grid metric case (MMI vs mmi)
#   - Forces unified-grid rebuild when grid_res changes
#   - Runs POINTS + AREAS
# ============================================================





if raw_vs_unified_validation:

    
    import os
    import types
    import numpy as np
    import pandas as pd
    
    # -----------------------------
    # 1) Create SHAKEuq instance (matches your current __init__)
    # -----------------------------
    # NOTE: your SHAKEuq.__init__(event_id, event_time=None, shakemap_folder=None, pager_folder=None, file_type=2)
    from modules.SHAKEtime import SHAKEtime
    
    shake = SHAKEuq(
        event_id=event_id,
        event_time=event_time,
        shakemap_folder=shakemap_folder,
        pager_folder=pager_folder,
        file_type=2,
    )
    
    # Optional: set any extra folders you like as attributes (won't break anything)
    shake.rupture_folder  = globals().get("rupture_folder", None)
    shake.stations_folder = globals().get("stations_folder", None)
    
    # Where figures/CSVs go
    out_root = globals().get("out_root", "export")
    os.makedirs(out_root, exist_ok=True)
    
    print("[OK] SHAKEuq instance created")
    print("  event_id:", shake.event_id)
    print("  shakemap_folder:", shake.shakemap_folder)
    print("  out_root:", out_root)
    
    # -----------------------------
    # 2) Targets (ensure schema)
    # -----------------------------
    points_city_uq = []
    for p in points_city:
        pp = dict(p)
        pp.setdefault("type", "point")
        pp["id"]  = str(pp.get("id", pp.get("name", "point")))
        pp["lat"] = float(pp["lat"])
        pp["lon"] = float(pp["lon"])
        points_city_uq.append(pp)
    
    print("[OK] Loaded", len(points_city_uq), "point targets")
    
    # -----------------------------
    # 3) Notebook-side hotfix:
    #    Ensure get_unified_grid always uses metric.lower()
    #    (prevents: "Metric 'MMI' not in vX; filling with NaN")
    # -----------------------------
    if hasattr(shake, "get_unified_grid"):
        _orig_get_unified_grid = shake.get_unified_grid
    
        def _get_unified_grid_lower(self, version_list, metric="mmi", grid_res=None,
                                    use_cache=False, interp_method="nearest", interp_kwargs=None):
            metric2 = str(metric).lower().strip()
            return _orig_get_unified_grid(
                version_list=version_list,
                metric=metric2,
                grid_res=grid_res,
                use_cache=use_cache,
                interp_method=interp_method,
                interp_kwargs=interp_kwargs,
            )
    
        shake.get_unified_grid = types.MethodType(_get_unified_grid_lower, shake)
        print("[OK] Hotfix applied: get_unified_grid() will use metric.lower()")
    
    # -----------------------------
    # 4) Helper: force unified-grid rebuild (VERY IMPORTANT)
    #    Because _uq_get_unified_for_versions calls get_unified_grid(use_cache=True)
    #    so changing grid_res won't matter unless we clear the cache.
    # -----------------------------
    def clear_unified_cache(obj):
        for a in ("_unified_grid", "_unified_lat2d", "_unified_lon2d"):
            if hasattr(obj, a):
                setattr(obj, a, None)
    
    # -----------------------------
    # 5) User controls
    # -----------------------------
    imt_val = "MMI"  # keep as "MMI" for your UQ API; hotfix will handle unified grid metric case
    version_list_val = [int(v) for v in version_list]  # version_list from your event block
    
    grid_size = None  # try 0.2, 0.5, 1.0 etc (degrees). Set None to use native smallest spacing.
    interp_method_val = "nearest"  # "nearest" | "linear" | "cubic" (scipy.griddata)
    
    raw_sample = "nearest"  # for POINTS: "nearest" | "bilinear"
    
    raw_area_max_cells = 25000
    raw_area_stride = None
    raw_area_random_state = 42
    raw_area_subsample_method = "auto"  # "auto" | "stride" | "random"
    
    band = True
    band_alpha = 0.18
    
    figsize_val = (12, 6)
    dpi_val = 220
    xrotation_val = 0
    
    export_table = True
    export_prefix_points = f"UQ-RawVsUnifiedCurves-POINTS-{event_id}"
    export_prefix_areas  = f"UQ-RawVsUnifiedCurves-AREAS-{event_id}"
    
    # For raw discovery in your locator:
    # base_folder should be 'event_data/SHAKEfetch' (your convention)
    base_folder_override = globals().get("base_folder", "event_data/SHAKEfetch")
    shakemap_folder_override = shakemap_folder
    
    # -----------------------------
    # 6) Quick RAW discovery check + RAW point sample sanity check
    # -----------------------------
    try:
        _v0 = int(version_list_val[0])
        _grid0 = shake._uq_find_shakemap_xml(_v0, which="grid",
                                             base_folder=base_folder_override,
                                             shakemap_folder=shakemap_folder_override)
        _unc0  = shake._uq_find_shakemap_xml(_v0, which="uncertainty",
                                             base_folder=base_folder_override,
                                             shakemap_folder=shakemap_folder_override)
        print("[RAW DISCOVERY]")
        print("  v0:", _v0)
        print("  grid:", _grid0)
        print("  unc :", _unc0)
    
        # raw sample at first point
        p0 = points_city_uq[0]
        raw0 = shake._uq_raw_sample_from_xml(_grid0, _unc0, p0["lat"], p0["lon"], imt=imt_val, sample=raw_sample)
        print("[RAW SAMPLE TEST]")
        print("  target:", p0["id"])
        print("  raw:", raw0)
    except Exception as e:
        print("[RAW CHECK FAILED]", repr(e))
    
    # -----------------------------
    # 7) Force unified rebuild at requested grid_size and report what you got
    # -----------------------------
    clear_unified_cache(shake)
    df_uni = shake.get_unified_grid(
        version_list=[str(v).zfill(3) for v in version_list_val],  # your builder expects strings
        metric=str(imt_val).lower(),                                # explicit here
        grid_res=grid_size,
        use_cache=False,
        interp_method=interp_method_val,
        interp_kwargs=None,
    )
    
    # unified shape check (nlat/nlon)
    lats = np.sort(df_uni["lat"].unique())
    lons = np.sort(df_uni["lon"].unique())
    dlat = float(np.median(np.diff(lats))) if len(lats) > 1 else np.nan
    dlon = float(np.median(np.diff(lons))) if len(lons) > 1 else np.nan
    
    print("[UNIFIED GRID]")
    print("  rows:", len(df_uni))
    print("  nlat:", len(lats), "nlon:", len(lons))
    print("  dlat≈", f"{dlat:.5f}°", "dlon≈", f"{dlon:.5f}°")
    print("  requested grid_res:", grid_size, "(if this doesn't match, the bbox/res logic may be overriding)")
    
    # sanity: check one interpolated column exists and is finite
    col0 = f"{str(imt_val).lower()}_v{version_list_val[0]}"
    if col0 in df_uni.columns:
        nfin = int(np.isfinite(df_uni[col0].values).sum())
        print(f"[OK] {col0} finite count:", nfin, "/", len(df_uni))
    else:
        print("[WARN] expected unified column missing:", col0)
    
    # -----------------------------
    # 8) Run RAW vs UNIFIED — POINTS
    # -----------------------------
    clear_unified_cache(shake)  # IMPORTANT: uq_compare will call unified again (with cache=True internally)
    df_raw_vs_uni_points = shake.uq_compare_raw_vs_unified_curves(
        version_list=version_list_val,
        imt=imt_val,
        points=points_city_uq,
        areas=None,
        agg="mean",
    
        base_folder=base_folder_override,
        shakemap_folder=shakemap_folder_override,
    
        raw_sample=raw_sample,
    
        grid_res=grid_size,
        interp_method=interp_method_val,
        interp_kwargs=None,
    
        figsize=figsize_val,
        dpi=dpi_val,
        xrotation=xrotation_val,
        show_grid=True,
        title=f"RAW vs UNIFIED — {imt_val} — {{target}} — POINTS | {event_id}",
        legend=True,
        legend_kwargs={"frameon": True},
        tight=True,
    
        output_path=out_root,
        save=True,
        save_formats=("png",),
        show=True,
    
        band=False,
        band_alpha=band_alpha,
    
        raw_area_max_cells=raw_area_max_cells,
        raw_area_stride=raw_area_stride,
        raw_area_random_state=raw_area_random_state,
        raw_area_subsample_method=raw_area_subsample_method,
    
        export_table=export_table,
        export_prefix=export_prefix_points,
    )
    
    print("[OK] RAW vs UNIFIED complete (POINTS)")
    display(df_raw_vs_uni_points.head(10))
    
    # -----------------------------
    # 9) Run RAW vs UNIFIED — AREAS (circles around points)
    # -----------------------------
    radius_km_val = 30.0
    areas_from_points = [
        {
            "id": f"{p['id']}_R{int(radius_km_val)}km",
            "type": "area",
            "kind": "circle",
            "lat": float(p["lat"]),
            "lon": float(p["lon"]),
            "radius_km": float(radius_km_val),
        }
        for p in points_city_uq
    ]
    
    clear_unified_cache(shake)  # IMPORTANT again
    df_raw_vs_uni_areas = shake.uq_compare_raw_vs_unified_curves(
        version_list=version_list_val,
        imt=imt_val,
        points=None,
        areas=areas_from_points,
        agg="mean",
    
        base_folder=base_folder_override,
        shakemap_folder=shakemap_folder_override,
    
        raw_sample=raw_sample,  # unused for areas
    
        grid_res=grid_size,
        interp_method=interp_method_val,
        interp_kwargs=None,
    
        figsize=figsize_val,
        dpi=dpi_val,
        xrotation=xrotation_val,
        show_grid=True,
        title=f"RAW vs UNIFIED — {imt_val} — {{target}} — AREAS(R={radius_km_val:.0f}km) | {event_id}",
        legend=True,
        legend_kwargs={"frameon": True},
        tight=True,
    
        output_path=out_root,
        save=True,
        save_formats=("png",),
        show=True,
    
        band=band,
        band_alpha=band_alpha,
    
        raw_area_max_cells=raw_area_max_cells,
        raw_area_stride=raw_area_stride,
        raw_area_random_state=raw_area_random_state,
        raw_area_subsample_method=raw_area_subsample_method,
    
        export_table=export_table,
        export_prefix=export_prefix_areas,
    )
    
    print("[OK] RAW vs UNIFIED complete (AREAS)")
    display(df_raw_vs_uni_areas.head(10))
    
    print("\n[FILES]")
    print("Plots saved under:", f"{out_root}/.../uq_plots/raw_vs_unified_curves/")
    if export_table:
        print("CSV (points):", f"{out_root}/SHAKEuq/{event_id}/uq/{export_prefix_points}-{imt_val}.csv")
        print("CSV (areas): ", f"{out_root}/SHAKEuq/{event_id}/uq/{export_prefix_areas}-{imt_val}.csv")


In [ ]:
print('Section Cleared!')

# End Of Script 

In [ ]:
import sys 
sys.exit()

In [ ]:
# ============================================================
# SHAKEuq Notebook Example: LOCAL TARGET UQ DECAY (PATCH DEMO)
# Compatible with YOUR PATCHED signatures:
#   - uq_plot_targets_decay(...)
#   - uq_plot_targets_mean_and_sigma(...)
# And ends with:
#   - uq_validate_unified_vs_raw(...)  (raw XML vs unified grid test)
# ============================================================

if target_uq_decay:

    from pathlib import Path
    import numpy as np
    import matplotlib.pyplot as plt

    # -----------------------------
    # 0) USER INPUTS
    # -----------------------------
    imt_choice = "MMI"  # "MMI" or "PGA"
    imtU = str(imt_choice).upper()

    # -----------------------------
    # 1) Normalize versions
    # -----------------------------
    version_list_int = [int(v) for v in version_list]
    v0 = int(version_list_int[0])
    v_last = int(version_list_int[-1])

    # -----------------------------
    # 2) Build dataset BEFORE plotting
    # -----------------------------
    uq_state = shake.uq_build_dataset(
        event_id=event_id,
        version_list=version_list_int,
        base_folder="./export",
        stations_folder=stations_folder,
        rupture_folder=rupture_folder,
        imts=("MMI", "PGA", "PGV", "PSA03"),
        grid_unify="intersection",
        resolution="finest",
        export=True,
    )

    # -----------------------------
    # 3) UQ knobs (used by update methods; ShakeMap sigma_total stays RAW)
    # -----------------------------
    sigma_total_from_shakemap = True  # per your decision

    sigma_aleatory_mmi = 0.40
    sigma_aleatory_pga = 0.35
    sigma_aleatory_psa03 = 0.35

    update_radius_km = 25.0
    kernel = "gaussian"
    kernel_scale_km = 20.0

    measurement_sigma_mmi = 0.30
    measurement_sigma_pga = 0.35

    ok_range_km = 60.0
    ok_variogram = "exponential"
    ok_nugget = 1e-6
    ok_sill = None
    ok_cap_sigma_to_prior = True

    mc_nsim = 1000
    mc_include_aleatory = True

    # -----------------------------
    # 4) Output / plot styling
    # -----------------------------
    out_root = "./export"
    Path(out_root).mkdir(parents=True, exist_ok=True)

    dpi = 300
    save_formats = ("png",)

    figsize = (24, 15)
    combined_figsize = (24, 15)
    xrotation = 0

    title_size  = 40
    label_size  = 38
    tick_size   = 34
    legend_size = 28

    legend_kwargs = {"loc": "best", "frameon": True}  # fontsize injected via legend_size
    ylog = False
    ymin = None
    ymax = None

    # -----------------------------
    # Helpers: per-IMT defaults
    # -----------------------------
    def sigma_aleatory_for_imt(imt):
        imtU0 = str(imt).upper()
        if imtU0 == "MMI":
            return sigma_aleatory_mmi
        if imtU0 in ("PGA", "PGV"):
            return sigma_aleatory_pga
        if imtU0 == "PSA03":
            return sigma_aleatory_psa03
        return None

    def measurement_sigma_for_imt(imt):
        imtU0 = str(imt).upper()
        if imtU0 == "MMI":
            return measurement_sigma_mmi
        if imtU0 == "PGA":
            return measurement_sigma_pga
        return 0.30

    # -----------------------------
    # Targets
    # -----------------------------
    points_tw = [
        {"id": "Taipei",      "lat": 25.0330, "lon": 121.5654},
        {"id": "Kaohsiung",   "lat": 22.6273, "lon": 120.3014},
        {"id": "Hualien",     "lat": 23.9872, "lon": 121.6015},
        {"id": "TW_Point_01", "lat": 24.9770, "lon": 120.9240},
    ]

    # -----------------------------
    # Footer text (embedded into title)
    # -----------------------------
    footer = (
        f"event={event_id} | prior=v{v0} | updateR={update_radius_km}km | "
        f"kernel={kernel}({kernel_scale_km}km) | sigma_total=RAW(SM)"
    )

    # ============================================================
    # 1) Uncertainty decay at each target (sigma_total) + combined
    # ============================================================
    df_tw_sigma = shake.uq_plot_targets_decay(
        version_list=version_list_int,
        imt=imtU,
        points=points_tw,
        areas=None,
        what="sigma_total",
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        agg="mean",
        global_stat=None,
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        # unified controls (match your uq_build_dataset defaults; keep as-is unless testing)
        grid_res=None,
        interp_method="nearest",
        interp_kwargs=None,

        # plotting kwargs supported by your patch
        figsize=figsize,
        dpi=dpi,
        ylog=ylog,
        ymin=ymin,
        ymax=ymax,
        xrotation=xrotation,
        show_title=True,
        title=f"{imtU} uncertainty decay @ {{target}} (sigma_total)\n{footer}",
        show_grid=True,
        legend=True,
        legend_kwargs=legend_kwargs,
        tight=True,

        label_size=label_size,
        tick_size=tick_size,
        title_size=title_size,
        legend_size=legend_size,
        xlabel="ShakeMap version",
        ylabel=f"σ_total ({imtU})",

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,

        plot_combined=True,
        combined_figsize=combined_figsize,
        combined_legend_ncol=2,

        audit=True,
        audit_output_path=out_root,
        audit_prefix=f"UQ-TargetAudit-{imtU}",
    )

    # ============================================================
    # 2) Mean + sigma panels at each target (3-panel per target)
    # ============================================================
    df_tw_panels = shake.uq_plot_targets_mean_and_sigma(
        version_list=version_list_int,
        imt=imtU,
        points=points_tw,
        areas=None,
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        agg="mean",
        global_stat=None,
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        grid_res=None,
        interp_method="nearest",
        interp_kwargs=None,

        figsize=(24, 18),
        dpi=dpi,
        ylog_sigma=False,
        xrotation=xrotation,
        show_title=True,
        title=f"Target series @ {{target}} — {imtU} (mean + σ_total + σ_ep)\n{footer}",

        legend=True,
        legend_kwargs=legend_kwargs,
        tight=True,

        label_size=label_size,
        tick_size=tick_size,
        title_size=title_size,
        legend_size=legend_size,
        xlabel="ShakeMap version",

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,

        plot_combined=True,

        audit=True,
        audit_output_path=out_root,
        audit_prefix=f"UQ-TargetAudit-Panels-{imtU}",
    )

    # ============================================================
    # 3) FINAL: RAW XML vs UNIFIED GRID VALIDATION TEST
    # ============================================================
    # This produces:
    #   - <out_root>/UQ-ValidateRawVsUnified-values.csv
    #   - <out_root>/UQ-ValidateRawVsUnified-errors.csv
    #   - optional error plots (per target/imt/param)
    #
    # IMPORTANT:
    # Provide base_folder/shakemap_folder if your XML files are not discoverable
    # from shake.uq_state["base_folder"].
    #
    df_val, df_err = shake.uq_validate_unified_vs_raw(
        version_list=version_list_int,
        points=points_tw,
        imts=(imtU,),                 # validate chosen IMT; or ("MMI","PGA")
        base_folder=None,             # set if needed (e.g., "./export/SHAKEuq")
        shakemap_folder=None,         # set if needed
        grid_res=None,
        interp_method="nearest",
        interp_kwargs=None,

        make_plots=True,
        figsize=(12, 5),
        dpi=200,
        xrotation=0,

        output_path=out_root,
        save=True,
        save_formats=("png",),

        export_tables=True,
        export_prefix="UQ-ValidateRawVsUnified",
    )

    print("[DONE] Local target patch demo complete.")
    print("IMT =", imtU, "| versions =", (v0, "→", v_last), "| outputs:", out_root)
    print("Validation tables:", f"{out_root}/UQ-ValidateRawVsUnified-values.csv",
          "and", f"{out_root}/UQ-ValidateRawVsUnified-errors.csv")


## Build UQ dataset (unified grid + priors + station parsing)

In [ ]:
if uncertainty_quantification: 
    
    # ============================================================
    # SHAKEuq Notebook Example: FULL UQ (GLOBAL + LOCAL + EXPORTS)
    # UPDATED for your *current plotting patch signatures* (the snippet you pasted):
    #
    # Key signature changes vs old examples:
    #   - plotting uses save=..., output_path=..., save_formats=..., figsize=...
    #   - xrotation (not rotation)
    #   - NO figure_kwargs / line_kwargs / band_kwargs in these upgraded plotters
    #     (styling is via figsize/font_sizes/grid_kwargs/legend_kwargs + post-editing ax)
    #   - uq_plot_uncertainty_decay has overlay_other_sigma=True option
    #
    # Also includes options to:
    #   - plot epistemic-only OR total-only
    #   - plot both on one figure and set y-axis log from notebook
    # ============================================================
    
    from pathlib import Path
    import numpy as np
    import matplotlib.pyplot as plt
    
    # -----------------------------
    # 1) Normalize versions
    # -----------------------------
    version_list_int = [int(v) for v in version_list]
    last_v = int(version_list_int[-1])
    
    # -----------------------------
    # 2) Create SHAKEuq instance
    # -----------------------------
    shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)
    
    # -----------------------------
    # 3) Build UQ dataset (unified grid + priors + station parsing)
    # -----------------------------
    uq_state = shake.uq_build_dataset(
        event_id=event_id,
        version_list=version_list_int,
        base_folder="./export",
        stations_folder=stations_folder,
        rupture_folder=rupture_folder,
        imts=("MMI", "PGA", "PGV", "PSA"),
        grid_unify="intersection",
        resolution="finest",
        export=True,
    )
    
    print("[OK] UQ dataset built")
    print("UQ base folder =", shake.uq_state["base_folder"])
    
    # Sanity audit table
    try:
        df_sanity = shake.uq_sanity_report()
        display(df_sanity)
    except Exception:
        print("[WARN] Could not display sanity report.")
    
    # -----------------------------
    # 4) Global configuration knobs
    # -----------------------------
    # Bayes update knobs
    update_radius_km = 25.0
    update_kernel = "gaussian"     # "gaussian" or "tophat"
    measurement_sigma = None       # optional extra obs noise
    
    # Aleatory assumptions (used in decomposition)
    sigma_total_from_shakemap = True
    sigma_aleatory_mmi = 0.40
    sigma_aleatory_pga = 0.35
    
    # Monte Carlo knobs
    nsim = 200
    correlation_length_km = 0.0    # try 10–30 km to add spatial correlation
    
    # Local metrics knobs
    station_radius_km = 20
    poe_threshold = 6
    poe_cut = 0.5
    
    # Output controls
    dpi = 300
    out_root = "./export"          # IMPORTANT: output_path in your patch expects a root like "./export"
    save_formats = ["png"]  # publishing-ready
    
    # -----------------------------
    # 4b) Publication-style plot kwargs (supported by your patch)
    # -----------------------------
    figsize = (24, 15)
    xrotation = 0
    
    font_sizes = {"title": 20, "labels": 40, "ticks": 40, "legend": 40}
    grid_kwargs = {"linestyle": "--", "alpha": 0.5}
    legend_kwargs = {"loc": "best", "fontsize": font_sizes["legend"], "frameon": True}
    
    # ============================================================
    # Helper options for decay plots (from notebook, no framework edits)
    # ============================================================
    
    def decay_epistemic_only(method="prior", imt="MMI", logy=False, overlay_other_sigma=False):
        fig, ax = shake.uq_plot_uncertainty_decay(
            imt=imt,
            which="epistemic",
            method=method,
            version_list=version_list_int,
            x_axis="version",
            xrotation=xrotation,
            show_title=True,
            figsize=figsize,
            font_sizes=font_sizes,
            grid=True,
            grid_kwargs=grid_kwargs,
            legend=True,
            legend_kwargs=legend_kwargs,
            output_path=out_root,
            save=True,
            save_formats=save_formats,
            dpi=dpi,
            show=True,
            overlay_other_sigma=overlay_other_sigma,
        )
        if logy:
            ax.set_yscale("log")
            fig.tight_layout()
        return fig, ax
    
    
    def decay_total_only(method="prior", imt="MMI", logy=False, overlay_other_sigma=False):
        fig, ax = shake.uq_plot_uncertainty_decay(
            imt=imt,
            which="total",
            method=method,
            version_list=version_list_int,
            x_axis="version",
            xrotation=xrotation,
            show_title=True,
            figsize=figsize,
            font_sizes=font_sizes,
            grid=True,
            grid_kwargs=grid_kwargs,
            legend=True,
            legend_kwargs=legend_kwargs,
            output_path=out_root,
            save=True,
            save_formats=save_formats,
            dpi=dpi,
            show=True,
            overlay_other_sigma=overlay_other_sigma,
        )
        if logy:
            ax.set_yscale("log")
            fig.tight_layout()
        return fig, ax
    
    
    def decay_both_on_one_figure(method="prior", imt="MMI", logy=True):
        # Use which="epistemic" (or "total") + overlay_other_sigma=True to get BOTH lines
        fig, ax = shake.uq_plot_uncertainty_decay(
            imt=imt,
            which="epistemic",               # primary line
            method=method,
            version_list=version_list_int,
            x_axis="version",
            xrotation=xrotation,
            show_title=True,
            figsize=figsize,
            font_sizes=font_sizes,
            grid=True,
            grid_kwargs=grid_kwargs,
            legend=True,
            legend_kwargs=legend_kwargs,
            output_path=out_root,
            save=True,
            save_formats=save_formats,
            dpi=dpi,
            show=True,
            overlay_other_sigma=True,        # adds total line automatically
        )
        if logy:
            ax.set_yscale("log")
            fig.tight_layout()
        return fig, ax



## Uncertainty decay maps 

In [ ]:
if uncertainty_quantification: 

    
    shake.uq_plot_shakemap_sigma_and_data_map(version=12, imt="PGA", which_sigma="total", 
                                              station_radius_km=10, output_path="./export", save_formats=["png"])
    
    shake.uq_plot_shakemap_sigma_and_data_map(version=13, imt="PGA", which_sigma="total", 
                                              station_radius_km=10, output_path="./export", save_formats=["png"])
    
    
    
    
    



In [ ]:
if uncertainty_quantification: 

    shake.uq_plot_shakemap_sigma_change_map(
        imt="PGA",
        which_sigma="total",   # true USGS sigma change
        v_from=1,
        v_to=12,figsize=figsize,

        mask_to_data_radius_km=10,  # focuses on “near data” area
        output_path="./export", save_formats=["png"]
    )

## GLOBAL PRIOR plots/metrics

In [ ]:
if uncertainty_quantification:
    # -----------------------------
    # 5) GLOBAL PRIOR plots/metrics
    # -----------------------------
    # (A) Epistemic-only decay
    decay_epistemic_only(method="prior", imt="MMI", logy=False, overlay_other_sigma=False)
    
    # (B) Total-only decay
    decay_total_only(method="prior", imt="MMI", logy=False, overlay_other_sigma=False)
    
    # (C) BOTH on one figure + log y
    decay_both_on_one_figure(method="prior", imt="MMI", logy=True)
    

In [ ]:
if uncertainty_quantification:
    
    # Prior mean PoE curve (supports overlay_other_sigma too)
    shake.uq_plot_metric_mean_poe(
        imt="MMI",
        threshold=poe_threshold,
        method="prior",
        version_list=version_list_int,
        which_sigma="total",            # choose the sigma used in PoE
        x_axis="version",
        xrotation=xrotation,
        show_title=True,
        figsize=figsize,
        font_sizes=font_sizes,
        grid=True,
        grid_kwargs=grid_kwargs,
        legend=True,
        legend_kwargs=legend_kwargs,
        output_path=out_root,
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        overlay_other_sigma=True,       # optional: compare PoE using total vs epistemic sigma
    )
    
    # Prior area (km²) where PoE >= cut (supports overlay_other_sigma too)
    shake.uq_plot_metric_area_ge_threshold(
        imt="MMI",
        threshold=poe_threshold,
        method="prior",
        version_list=version_list_int,
        which_sigma="total",
        x_axis="version",
        xrotation=xrotation,
        poe_cut=poe_cut,                # NOTE: poe_cut is explicit in your upgraded signature
        show_title=True,
        figsize=figsize,
        font_sizes=font_sizes,
        grid=True,
        grid_kwargs=grid_kwargs,
        legend=True,
        legend_kwargs=legend_kwargs,
        output_path=out_root,
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        overlay_other_sigma=True,
    )


### BAYES UPDATE (per version posterior)

In [ ]:
if uncertainty_quantification:
    # -----------------------------
    # 6) BAYES UPDATE (per version posterior)
    # -----------------------------
    bayes_mmi = shake.uq_bayes_update(
        version_list=version_list_int,
        imt="MMI",
        update_radius_km=update_radius_km,
        update_kernel=update_kernel,
        sigma_aleatory=sigma_aleatory_mmi,
        sigma_total_from_shakemap=sigma_total_from_shakemap,
        measurement_sigma=measurement_sigma,
        export=True,
        make_audit=True,
    )
    print("[OK] BayesUpdate(MMI) finished")
    print("Example audit last version:", bayes_mmi["audit"].get(last_v, {}))
    
    # Optional: BayesUpdate on PGA (only meaningful if stationlist has real PGA values)
    try:
        bayes_pga = shake.uq_bayes_update(
            version_list=version_list_int,
            imt="PGA",
            update_radius_km=update_radius_km,
            update_kernel=update_kernel,
            sigma_aleatory=sigma_aleatory_pga,
            sigma_total_from_shakemap=sigma_total_from_shakemap,
            export=True,
            make_audit=True,
        )
        print("[OK] BayesUpdate(PGA) finished")
    except Exception as e:
        print("[WARN] BayesUpdate(PGA) skipped/failed:", e)
    


## GLOBAL POSTERIOR plots/metrics (BayesUpdate)

In [ ]:
if uncertainty_quantification:
    # -----------------------------
    # 7) GLOBAL POSTERIOR plots/metrics (BayesUpdate)
    # -----------------------------
    # (A) Epistemic-only decay
    decay_epistemic_only(method="bayesupdate", imt="MMI", logy=False, overlay_other_sigma=False)
    
    # (B) Total-only decay
    decay_total_only(method="bayesupdate", imt="MMI", logy=False, overlay_other_sigma=False)
    
    # (C) BOTH on one figure + log y
    decay_both_on_one_figure(method="bayesupdate", imt="MMI", logy=True)
    
    shake.uq_plot_metric_mean_poe(
        imt="MMI",
        threshold=poe_threshold,
        method="bayesupdate",
        version_list=version_list_int,
        which_sigma="total",
        x_axis="version",
        xrotation=xrotation,
        show_title=True,
        figsize=figsize,
        font_sizes=font_sizes,
        grid=True,
        grid_kwargs=grid_kwargs,
        legend=True,
        legend_kwargs=legend_kwargs,
        output_path=out_root,
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        overlay_other_sigma=True,
    )
    
    shake.uq_plot_metric_area_ge_threshold(
        imt="MMI",
        threshold=poe_threshold,
        method="bayesupdate",
        version_list=version_list_int,
        which_sigma="total",
        x_axis="version",
        xrotation=xrotation,
        poe_cut=poe_cut,
        show_title=True,
        figsize=figsize,
        font_sizes=font_sizes,
        grid=True,
        grid_kwargs=grid_kwargs,
        legend=True,
        legend_kwargs=legend_kwargs,
        output_path=out_root,
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        overlay_other_sigma=True,
    )


### HIERARCHICAL pooling (one pooled posterior)

In [ ]:
if uncertainty_quantification: 
    # -----------------------------
    # 8) HIERARCHICAL pooling (one pooled posterior)
    # -----------------------------
    hier_mmi = shake.uq_hierarchical(
        version_list=version_list_int,
        imt="MMI",
        update_radius_km=update_radius_km,
        update_kernel=update_kernel,
        sigma_aleatory=sigma_aleatory_mmi,
        sigma_total_from_shakemap=sigma_total_from_shakemap,
        export=True,
    )
    print("[OK] Hierarchical(MMI) finished")
    print("Hier audit:", hier_mmi.get("audit", {}))
    
    # Hierarchical decay plots (pooled posterior loaded from uq_results)
    decay_epistemic_only(method="hierarchical", imt="MMI", logy=False, overlay_other_sigma=False)
    decay_total_only(method="hierarchical", imt="MMI", logy=False, overlay_other_sigma=False)
    decay_both_on_one_figure(method="hierarchical", imt="MMI", logy=True)


### MONTE CARLO (prior propagation)

In [ ]:
if uncertainty_quantification:    
    # -----------------------------
    # 9) MONTE CARLO (prior propagation)
    # -----------------------------
    mc_total = shake.uq_monte_carlo(
        version_list=version_list_int,
        imt="MMI",
        nsim=nsim,
        which_sigma="total",
        sigma_aleatory=sigma_aleatory_mmi,
        sigma_total_from_shakemap=sigma_total_from_shakemap,
        correlation_length_km=correlation_length_km,
        export=True,
        seed=1234,
    )
    print("[OK] MonteCarlo total done")
    
    mc_ep = shake.uq_monte_carlo(
        version_list=version_list_int,
        imt="MMI",
        nsim=nsim,
        which_sigma="epistemic",
        sigma_aleatory=sigma_aleatory_mmi,
        sigma_total_from_shakemap=sigma_total_from_shakemap,
        correlation_length_km=correlation_length_km,
        export=True,
        seed=5678,
    )
    print("[OK] MonteCarlo epistemic done")


### LOCAL METRICS + LOCAL DECAY PLOT (posterior)

## LOCAL MAPS: sigma reduction (Δσ = σ_prior - σ_post)

### MMI 

In [ ]:
if uncertainty_quantification:
    # -----------------------------
    # 11) LOCAL MAPS: sigma reduction (Δσ = σ_prior - σ_post)
    # -----------------------------
    # Single map for the last version (Cartopy-based in your patch)
    shake.uq_plot_sigma_reduction_map(
        version=last_v,
        imt="MMI",
        which_sigma="epistemic",
        method="bayesupdate",
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=True,
        obs_size=18.0,
        figsize=(10, 8),
        show_title=True,
        output_path=out_root,
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        plot_colorbar=True,
        cbar_label="Δσ (epistemic) = σ_prior − σ_post",
    )
    
    # Batch export for all versions
    shake.uq_export_sigma_reduction_maps(
        imt="MMI",
        which_sigma="epistemic",
        method="bayesupdate",
        version_list=version_list_int,
        cmap="viridis",
        vmin=None,
        vmax=None,
        output_path=out_root,
        save_formats=save_formats,
        dpi=dpi,
        figsize=(10, 8),
        show_obs=True,
        obs_size=14.0,
        plot_colorbar=True,
    )
    
    print("\n[DONE] Full UQ workflow complete.")
    print("Outputs under:", Path(out_root) / "SHAKEuq" / event_id / "uq")


### PGA 

In [ ]:
if uncertainty_quantification:
    # -----------------------------
    # 11) LOCAL MAPS: sigma reduction (Δσ = σ_prior - σ_post)
    # -----------------------------
    # Single map for the last version (Cartopy-based in your patch)
    shake.uq_plot_sigma_reduction_map(
        version=last_v,
        imt="PGA",
        which_sigma="epistemic",
        method="bayesupdate",
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=True,
        obs_size=18.0,
        figsize=(10, 8),
        show_title=True,
        output_path=out_root,
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        plot_colorbar=True,
        cbar_label="Δσ (epistemic) = σ_prior − σ_post",
    )
    
    # Batch export for all versions
    shake.uq_export_sigma_reduction_maps(
        imt="PGA",
        which_sigma="epistemic",
        method="bayesupdate",
        version_list=version_list_int,
        cmap="viridis",
        vmin=None,
        vmax=None,
        output_path=out_root,
        save_formats=save_formats,
        dpi=dpi,
        figsize=(10, 8),
        show_obs=True,
        obs_size=14.0,
        plot_colorbar=True,
    )
    
    print("\n[DONE] Full UQ workflow complete.")
    print("Outputs under:", Path(out_root) / "SHAKEuq" / event_id / "uq")

## FIRST→FINAL UNCERTAINTY CHANGE + NET GAIN/LOSS DECOMPOSITION

In [ ]:
### MMI 

In [ ]:
if uncertainty_quantification:
    # -----------------------------
    # 12) FIRST→FINAL UNCERTAINTY CHANGE + NET GAIN/LOSS DECOMPOSITION
    # -----------------------------
    # Requires:
    #   - shake.uq_build_dataset(...) already run
    #   - shake.uq_bayes_update(..., export=True) already run (or hierarchical export if you use that)
    
    v_first = int(version_list_int[0])
    v_final = int(version_list_int[-1])
    
    # (A) Net posterior uncertainty change map (first -> final)
    #     Δσ_post = σ_post(v_first) − σ_post(v_final)
    # Positive values => uncertainty DECREASED by the final version.
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="MMI",
        which_sigma="epistemic",          # or "total"
        posterior_method="bayesupdate",   # or "hierarchical"
        kind="post",                      # "post"|"prior"|"reduction"|"residual"
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=True,
        obs_size=14.0,
        figsize=figsize,
        show_title=True,
        output_path="./export",           # optional; uses output_path/SHAKEuq/<event_id>/uq/...
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        plot_colorbar=True,
    )
    
    # (B) Prior evolution map (first -> final)
    #     Δσ_prior = σ_prior(v_first) − σ_prior(v_final)
    # Positive values => prior uncertainty DECREASED as the ShakeMap procedure evolved.
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="MMI",
        which_sigma="total",
        posterior_method="bayesupdate",
        kind="prior",
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=False,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    # (C) Update-effect evolution map (first -> final)
    #     reduction(v) = σ_prior(v) − σ_post(v)
    #     Δreduction = reduction(v_final) − reduction(v_first)
    # Positive values => observations reduce MORE in the final version than in the first.
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="MMI",
        which_sigma="epistemic",
        posterior_method="bayesupdate",
        kind="reduction",
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=True,
        obs_size=14.0,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    # (D) One publication figure: 3-panel decomposition
    #     Panel 1: Δσ_post (net posterior change)
    #     Panel 2: Δσ_prior (model/procedure change)
    #     Panel 3: Δreduction (data/update-effect change)
    shake.uq_plot_net_uncertainty_decomposition_first_to_final(
        imt="MMI",
        which_sigma="epistemic",
        posterior_method="bayesupdate",
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    # (Optional) Residual map (diagnostic): should be ~0 everywhere
    # residual = Δσ_post − (Δσ_prior − Δreduction)
    # If you see structure here, something is inconsistent (or numerical issues).
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="MMI",
        which_sigma="epistemic",
        posterior_method="bayesupdate",
        kind="residual",
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=False,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    print("[OK] Example 12 complete — first→final maps + decomposition exported.")


### PGA

In [ ]:
if uncertainty_quantification:
    # -----------------------------
    # 12) FIRST→FINAL UNCERTAINTY CHANGE + NET GAIN/LOSS DECOMPOSITION
    # -----------------------------
    # Requires:
    #   - shake.uq_build_dataset(...) already run
    #   - shake.uq_bayes_update(..., export=True) already run (or hierarchical export if you use that)
    
    v_first = int(version_list_int[0])
    v_final = int(version_list_int[-1])
    
    # (A) Net posterior uncertainty change map (first -> final)
    #     Δσ_post = σ_post(v_first) − σ_post(v_final)
    # Positive values => uncertainty DECREASED by the final version.
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="PGA",
        which_sigma="epistemic",          # or "total"
        posterior_method="bayesupdate",   # or "hierarchical"
        kind="post",                      # "post"|"prior"|"reduction"|"residual"
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=True,
        obs_size=14.0,
        figsize=figsize,
        show_title=True,
        output_path="./export",           # optional; uses output_path/SHAKEuq/<event_id>/uq/...
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
        plot_colorbar=True,
    )
    
    # (B) Prior evolution map (first -> final)
    #     Δσ_prior = σ_prior(v_first) − σ_prior(v_final)
    # Positive values => prior uncertainty DECREASED as the ShakeMap procedure evolved.
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="PGA",
        which_sigma="total",
        posterior_method="bayesupdate",
        kind="prior",
        first_version=v_first,
        final_version=12,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=False,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    # (C) Update-effect evolution map (first -> final)
    #     reduction(v) = σ_prior(v) − σ_post(v)
    #     Δreduction = reduction(v_final) − reduction(v_first)
    # Positive values => observations reduce MORE in the final version than in the first.
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="PGA",
        which_sigma="epistemic",
        posterior_method="bayesupdate",
        kind="reduction",
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=True,
        obs_size=14.0,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    # (D) One publication figure: 3-panel decomposition
    #     Panel 1: Δσ_post (net posterior change)
    #     Panel 2: Δσ_prior (model/procedure change)
    #     Panel 3: Δreduction (data/update-effect change)
    shake.uq_plot_net_uncertainty_decomposition_first_to_final(
        imt="PGA",
        which_sigma="epistemic",
        posterior_method="bayesupdate",
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    # (Optional) Residual map (diagnostic): should be ~0 everywhere
    # residual = Δσ_post − (Δσ_prior − Δreduction)
    # If you see structure here, something is inconsistent (or numerical issues).
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="PGA",
        which_sigma="epistemic",
        posterior_method="bayesupdate",
        kind="residual",
        first_version=v_first,
        final_version=v_final,
        cmap="viridis",
        vmin=None,
        vmax=None,
        show_obs=False,
        figsize=figsize,
        output_path="./export",
        save=True,
        save_formats=save_formats,
        dpi=dpi,
        show=True,
    )
    
    print("[OK] Example 12 complete — first→final maps + decomposition exported.")

In [ ]:
if uncertainty_quantification:
    
    
    shake.uq_plot_uncertainty_change_first_to_final_map(
        imt="PGA",
        which_sigma="total",
        posterior_method="bayesupdate",  # irrelevant for kind="prior"
        kind="prior",
        first_version=v_first,
        final_version=12,
        show_obs=True,
    )


## Compare all methods, epistemic, linear y

In [ ]:
if uncertainty_quantification:
    # Compare all methods, epistemic, linear y
    shake.uq_plot_uncertainty_decay_compare(
        imt="MMI",
        which="epistemic",
        methods=("prior", "bayesupdate", "hierarchical", "montecarlo"),
        version_list=version_list_int,
        x_axis="version",
        xrotation=0,
        ylog=False,
        output_path="./export",
                save_formats=save_formats,

        save=True,
        dpi=300,
        show=True,
    )
    
    # Same but TOTAL and log y-axis
    shake.uq_plot_uncertainty_decay_compare(
        imt="MMI",
        which="total",
        methods=("prior", "bayesupdate", "hierarchical", "montecarlo"),
        version_list=version_list_int,
        x_axis="version",
        xrotation=0,
        ylog=False,           # <- log axis
        output_path="./export",  save_formats=save_formats,

        save=True,
        dpi=300,
        show=True,
    )


## TARGET + GLOBAL UQ DECAY (PATCH DEMO)

In [ ]:
### target location Decay 

In [ ]:
# ============================================================
# SHAKEuq Notebook Example: LOCAL TARGET UQ DECAY (PATCH DEMO)
# Compatible with YOUR PATCHED signatures:
#   - uq_plot_targets_decay(...)
#   - uq_plot_targets_mean_and_sigma(...)
# And ends with:
#   - uq_validate_unified_vs_raw(...)  (raw XML vs unified grid test)
# ============================================================

if target_uq_decay:

    from pathlib import Path
    import numpy as np
    import matplotlib.pyplot as plt

    # -----------------------------
    # 0) USER INPUTS
    # -----------------------------
    imt_choice = "MMI"  # "MMI" or "PGA"
    imtU = str(imt_choice).upper()

    # -----------------------------
    # 1) Normalize versions
    # -----------------------------
    version_list_int = [int(v) for v in version_list]
    v0 = int(version_list_int[0])
    v_last = int(version_list_int[-1])

    # -----------------------------
    # 2) Build dataset BEFORE plotting
    # -----------------------------
    uq_state = shake.uq_build_dataset(
        event_id=event_id,
        version_list=version_list_int,
        base_folder="./export",
        stations_folder=stations_folder,
        rupture_folder=rupture_folder,
        imts=("MMI", "PGA", "PGV", "PSA03"),
        grid_unify="intersection",
        resolution="finest",
        export=True,
    )

    # -----------------------------
    # 3) UQ knobs (used by update methods; ShakeMap sigma_total stays RAW)
    # -----------------------------
    sigma_total_from_shakemap = True  # per your decision

    sigma_aleatory_mmi = 0.40
    sigma_aleatory_pga = 0.35
    sigma_aleatory_psa03 = 0.35

    update_radius_km = 25.0
    kernel = "gaussian"
    kernel_scale_km = 20.0

    measurement_sigma_mmi = 0.30
    measurement_sigma_pga = 0.35

    ok_range_km = 60.0
    ok_variogram = "exponential"
    ok_nugget = 1e-6
    ok_sill = None
    ok_cap_sigma_to_prior = True

    mc_nsim = 1000
    mc_include_aleatory = True

    # -----------------------------
    # 4) Output / plot styling
    # -----------------------------
    out_root = "./export"
    Path(out_root).mkdir(parents=True, exist_ok=True)

    dpi = 300
    save_formats = ("png",)

    figsize = (24, 15)
    combined_figsize = (24, 15)
    xrotation = 0

    title_size  = 40
    label_size  = 38
    tick_size   = 34
    legend_size = 28

    legend_kwargs = {"loc": "best", "frameon": True}  # fontsize injected via legend_size
    ylog = False
    ymin = None
    ymax = None

    # -----------------------------
    # Helpers: per-IMT defaults
    # -----------------------------
    def sigma_aleatory_for_imt(imt):
        imtU0 = str(imt).upper()
        if imtU0 == "MMI":
            return sigma_aleatory_mmi
        if imtU0 in ("PGA", "PGV"):
            return sigma_aleatory_pga
        if imtU0 == "PSA03":
            return sigma_aleatory_psa03
        return None

    def measurement_sigma_for_imt(imt):
        imtU0 = str(imt).upper()
        if imtU0 == "MMI":
            return measurement_sigma_mmi
        if imtU0 == "PGA":
            return measurement_sigma_pga
        return 0.30

    # -----------------------------
    # Targets
    # -----------------------------
    points_tw = [
        {"id": "Taipei",      "lat": 25.0330, "lon": 121.5654},
        {"id": "Kaohsiung",   "lat": 22.6273, "lon": 120.3014},
        {"id": "Hualien",     "lat": 23.9872, "lon": 121.6015},
        {"id": "TW_Point_01", "lat": 24.9770, "lon": 120.9240},
    ]

    # -----------------------------
    # Footer text (embedded into title)
    # -----------------------------
    footer = (
        f"event={event_id} | prior=v{v0} | updateR={update_radius_km}km | "
        f"kernel={kernel}({kernel_scale_km}km) | sigma_total=RAW(SM)"
    )

    # ============================================================
    # 1) Uncertainty decay at each target (sigma_total) + combined
    # ============================================================
    df_tw_sigma = shake.uq_plot_targets_decay(
        version_list=version_list_int,
        imt=imtU,
        points=points_tw,
        areas=None,
        what="sigma_total",
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        agg="mean",
        global_stat=None,
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        # unified controls (match your uq_build_dataset defaults; keep as-is unless testing)
        grid_res=None,
        interp_method="nearest",
        interp_kwargs=None,

        # plotting kwargs supported by your patch
        figsize=figsize,
        dpi=dpi,
        ylog=ylog,
        ymin=ymin,
        ymax=ymax,
        xrotation=xrotation,
        show_title=True,
        title=f"{imtU} uncertainty decay @ {{target}} (sigma_total)\n{footer}",
        show_grid=True,
        legend=True,
        legend_kwargs=legend_kwargs,
        tight=True,

        label_size=label_size,
        tick_size=tick_size,
        title_size=title_size,
        legend_size=legend_size,
        xlabel="ShakeMap version",
        ylabel=f"σ_total ({imtU})",

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,

        plot_combined=True,
        combined_figsize=combined_figsize,
        combined_legend_ncol=2,

        audit=True,
        audit_output_path=out_root,
        audit_prefix=f"UQ-TargetAudit-{imtU}",
    )

    # ============================================================
    # 2) Mean + sigma panels at each target (3-panel per target)
    # ============================================================
    df_tw_panels = shake.uq_plot_targets_mean_and_sigma(
        version_list=version_list_int,
        imt=imtU,
        points=points_tw,
        areas=None,
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        agg="mean",
        global_stat=None,
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        grid_res=None,
        interp_method="nearest",
        interp_kwargs=None,

        figsize=(24, 18),
        dpi=dpi,
        ylog_sigma=False,
        xrotation=xrotation,
        show_title=True,
        title=f"Target series @ {{target}} — {imtU} (mean + σ_total + σ_ep)\n{footer}",

        legend=True,
        legend_kwargs=legend_kwargs,
        tight=True,

        label_size=label_size,
        tick_size=tick_size,
        title_size=title_size,
        legend_size=legend_size,
        xlabel="ShakeMap version",

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,

        plot_combined=True,

        audit=True,
        audit_output_path=out_root,
        audit_prefix=f"UQ-TargetAudit-Panels-{imtU}",
    )

    # ============================================================
    # 3) FINAL: RAW XML vs UNIFIED GRID VALIDATION TEST
    # ============================================================
    # This produces:
    #   - <out_root>/UQ-ValidateRawVsUnified-values.csv
    #   - <out_root>/UQ-ValidateRawVsUnified-errors.csv
    #   - optional error plots (per target/imt/param)
    #
    # IMPORTANT:
    # Provide base_folder/shakemap_folder if your XML files are not discoverable
    # from shake.uq_state["base_folder"].
    #
    df_val, df_err = shake.uq_validate_unified_vs_raw(
        version_list=version_list_int,
        points=points_tw,
        imts=(imtU,),                 # validate chosen IMT; or ("MMI","PGA")
        base_folder=None,             # set if needed (e.g., "./export/SHAKEuq")
        shakemap_folder=None,         # set if needed
        grid_res=None,
        interp_method="nearest",
        interp_kwargs=None,

        make_plots=True,
        figsize=(12, 5),
        dpi=200,
        xrotation=0,

        output_path=out_root,
        save=True,
        save_formats=("png",),

        export_tables=True,
        export_prefix="UQ-ValidateRawVsUnified",
    )

    print("[DONE] Local target patch demo complete.")
    print("IMT =", imtU, "| versions =", (v0, "→", v_last), "| outputs:", out_root)
    print("Validation tables:", f"{out_root}/UQ-ValidateRawVsUnified-values.csv",
          "and", f"{out_root}/UQ-ValidateRawVsUnified-errors.csv")


### Global target 

In [ ]:
# ============================================================
# SHAKEuq Notebook Example: GLOBAL UQ DECAY (PATCH DEMO)
# NEW patch functions:
#   - uq_extract_target_series(...)
#   - uq_plot_targets_decay(...)
#
# Adds:
#   - IMT switch: "MMI" or "PGA"
#   - Label/tick/title/legend sizes via plot kwargs
# ============================================================

if target_uq_decay:

    from pathlib import Path
    import numpy as np
    import matplotlib.pyplot as plt

    # -----------------------------
    # 0) USER INPUT: Choose IMT
    # -----------------------------
    # Options: "MMI" or "PGA"
    imt_choice = "MMI"   # <-- change to "PGA" to track PGA instead

    # -----------------------------
    # 1) Normalize versions
    # -----------------------------
    version_list_int = [int(v) for v in version_list]
    v0 = int(version_list_int[0])      # baseline prior version
    v_last = int(version_list_int[-1])

    # -----------------------------
    # 2) Create SHAKEuq instance
    # -----------------------------
    shake = SHAKEuq(event_id, event_time, shakemap_folder, pager_folder, file_type=2)

    # -----------------------------
    # 3) Build UQ dataset (recommended)
    # -----------------------------
    uq_state = shake.uq_build_dataset(
        event_id=event_id,
        version_list=version_list_int,
        base_folder="./export/SHAKEuq",
        stations_folder=stations_folder,
        rupture_folder=rupture_folder,
        imts=("MMI", "PGA", "PGV", "PSA03"),
        grid_unify="intersection",
        resolution="finest",
        export=True,
    )

    print("[OK] UQ dataset built")
    print("UQ base folder =", shake.uq_state.get("base_folder", None))

    # -----------------------------
    # 4) UQ model knobs
    # -----------------------------
    # Interpretation of ShakeMap sigma
    sigma_total_from_shakemap = True

    # Aleatory floors (tune if needed)
    sigma_aleatory_mmi = 0.40
    sigma_aleatory_pga = 0.35

    # Update knobs
    update_radius_km = 25.0
    kernel = "gaussian"          # "gaussian" or "tophat"
    kernel_scale_km = 20.0

    # Additional observation noise term (working space)
    # For MMI, 0.30 is typical-ish; for PGA (log space) you may want smaller/larger depending on your workflow.
    measurement_sigma_mmi = 0.30
    measurement_sigma_pga = 0.35

    # Kriging knobs
    ok_range_km = 60.0
    ok_variogram = "exponential"
    ok_nugget = 1e-6
    ok_sill = None
    ok_cap_sigma_to_prior = True

    # Monte Carlo knobs
    mc_nsim = 1000
    mc_include_aleatory = True

    # -----------------------------
    # 5) Output + plot styling knobs
    # -----------------------------
    out_root = "./export"
    Path(out_root).mkdir(parents=True, exist_ok=True)
    dpi = 300
    save_formats = ("png",)   # add "pdf" if needed

    figsize = (24, 15)
    xrotation = 0

    # NEW: sizes (use these everywhere)
    title_size = 40
    label_size = 38
    tick_size  = 34
    legend_size = 28

    # Legend placement
    legend_kwargs = {"loc": "best", "frameon": True}

    # Optional line/grid tuning (supported by updated plotters)
    line_kwargs = {"linewidth": 2.5}
    grid_kwargs = {"alpha": 0.35, "linestyle": "--"}

    # -----------------------------
    # 6) Helper: per-IMT defaults
    # -----------------------------
    def sigma_aleatory_for_imt(imt):
        imtU = str(imt).upper()
        if imtU == "MMI":
            return sigma_aleatory_mmi
        if imtU == "PGA":
            return sigma_aleatory_pga
        return None

    def measurement_sigma_for_imt(imt):
        imtU = str(imt).upper()
        if imtU == "MMI":
            return measurement_sigma_mmi
        if imtU == "PGA":
            return measurement_sigma_pga
        return 0.30

    # -----------------------------
    # 7) Convenience labels
    # -----------------------------
    imtU = str(imt_choice).upper()
    ylab_sigma = f"σ_total ({imtU})"
    ylab_delta = f"Δσ_total ({imtU})"

    # ============================================================
    # A) GLOBAL PLOTS: mean / median / delta
    # ============================================================

    # ---- 1) Global MEAN ----
    df_global_mean = shake.uq_plot_targets_decay(
        version_list=version_list_int,
        imt=imtU,
        global_stat="mean",
        what="sigma_total",
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        figsize=figsize,
        dpi=dpi,
        xrotation=xrotation,
        legend_kwargs=legend_kwargs,

        # NEW sizing + labels
        xlabel="ShakeMap version",
        ylabel=ylab_sigma,
        title_size=title_size,
        label_size=label_size,
        tick_size=tick_size,
        legend_size=legend_size,
        line_kwargs=line_kwargs,
        grid_kwargs=grid_kwargs,

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,
        plot_combined=False,
        title=f"GLOBAL mean (whole grid) — {imtU} total uncertainty",
    )

    # ---- 2) Global MEDIAN ----
    df_global_median = shake.uq_plot_targets_decay(
        version_list=version_list_int,
        imt=imtU,
        global_stat="median",
        what="sigma_total",
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        figsize=figsize,
        dpi=dpi,
        xrotation=xrotation,
        legend_kwargs=legend_kwargs,

        # NEW sizing + labels
        xlabel="ShakeMap version",
        ylabel=ylab_sigma,
        title_size=title_size,
        label_size=label_size,
        tick_size=tick_size,
        legend_size=legend_size,
        line_kwargs=line_kwargs,
        grid_kwargs=grid_kwargs,

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,
        plot_combined=False,
        title=f"GLOBAL median (whole grid) — {imtU} total uncertainty",
    )

    # ---- 3) Delta MEAN (vs baseline v0) ----
    df_global_delta_mean = shake.uq_plot_targets_decay(
        version_list=version_list_int,
        imt=imtU,
        global_stat="delta mean",
        what="delta mean",   # alias -> delta_sigma_total
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        figsize=figsize,
        dpi=dpi,
        xrotation=xrotation,
        legend_kwargs=legend_kwargs,

        # NEW sizing + labels
        xlabel="ShakeMap version",
        ylabel=ylab_delta,
        title_size=title_size,
        label_size=label_size,
        tick_size=tick_size,
        legend_size=legend_size,
        line_kwargs=line_kwargs,
        grid_kwargs=grid_kwargs,

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,
        plot_combined=False,
        title=f"GLOBAL delta-mean uncertainty vs v{v0} — {imtU} (Δσ_total)",
    )

    # ---- 4) Delta MEDIAN (optional) ----
    df_global_delta_median = shake.uq_plot_targets_decay(
        version_list=version_list_int,
        imt=imtU,
        global_stat="delta median",
        what="delta median",  # alias -> delta_sigma_total
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        figsize=figsize,
        dpi=dpi,
        xrotation=xrotation,
        legend_kwargs=legend_kwargs,

        # NEW sizing + labels
        xlabel="ShakeMap version",
        ylabel=ylab_delta,
        title_size=title_size,
        label_size=label_size,
        tick_size=tick_size,
        legend_size=legend_size,
        line_kwargs=line_kwargs,
        grid_kwargs=grid_kwargs,

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,
        plot_combined=False,
        title=f"GLOBAL delta-median uncertainty vs v{v0} — {imtU} (Δσ_total)",
    )

    print("[DONE] Global patch demo complete for IMT =", imtU)
    print("Check outputs in:", out_root)


## Target Decay

In [ ]:
"""
# ============================================================
# SHAKEuq Notebook Example: LOCAL TARGET UQ DECAY (PATCH DEMO)
# For:
#   - uq_plot_targets_decay(...)
#   - uq_plot_targets_mean_and_sigma(...)
#
# ============================================================

if target_uq_decay:

    from pathlib import Path
    import numpy as np
    import matplotlib.pyplot as plt

    # -----------------------------
    # 0) USER INPUT: Choose IMT
    # -----------------------------
    # Options: "MMI" or "PGA"
    imt_choice = "PGA"   # <-- change to "PGA" for PGA tracking
    imtU = str(imt_choice).upper()

    # -----------------------------
    # 1) Normalize versions
    # -----------------------------
    version_list_int = [int(v) for v in version_list]
    v0 = int(version_list_int[0])      # baseline prior version
    v_last = int(version_list_int[-1])

    # -----------------------------
    # 4) Global configuration knobs (models + decomposition)
    # -----------------------------
    sigma_total_from_shakemap = True

    # Aleatory floors (tune per IMT)
    sigma_aleatory_mmi = 0.40
    sigma_aleatory_pga = 0.35
    sigma_aleatory_psa03 = 0.35

    # Bayes/Hierarchical update knobs
    update_radius_km = 25.0
    kernel = "gaussian"                 # "gaussian" or "tophat"
    kernel_scale_km = 20.0

    # Measurement noise (working space)
    # (you can tune PGA separately if desired)
    measurement_sigma_mmi = 0.30
    measurement_sigma_pga = 0.35

    # Kriging knobs
    ok_range_km = 60.0
    ok_variogram = "exponential"
    ok_nugget = 1e-6
    ok_sill = None
    ok_cap_sigma_to_prior = True

    # Monte Carlo knobs
    mc_nsim = 1000
    mc_include_aleatory = True

    # -----------------------------
    # 5) Output / plot styling
    # -----------------------------
    out_root = "./export"
    Path(out_root).mkdir(parents=True, exist_ok=True)
    dpi = 300
    save_formats = ("png",)   # add "pdf" if needed

    figsize = (24, 15)
    combined_figsize = (24, 15)
    xrotation = 0

    # NEW: sizes (set once, applies to both plotters)
    title_size  = 40
    label_size  = 38
    tick_size   = 34
    legend_size = 28

    legend_kwargs = {"loc": "best", "frameon": True}  # fontsize set via legend_size below

    # Optional line/grid styling
    line_kwargs = {"linewidth": 2.5}
    grid_kwargs = {"alpha": 0.35, "linestyle": "--"}

    # -----------------------------
    # Helpers: per-IMT defaults
    # -----------------------------
    def sigma_aleatory_for_imt(imt):
        imtU0 = str(imt).upper()
        if imtU0 == "MMI":
            return sigma_aleatory_mmi
        if imtU0 in ("PGA", "PGV"):
            return sigma_aleatory_pga
        if imtU0 in ("PSA03",):
            return sigma_aleatory_psa03
        return None

    def measurement_sigma_for_imt(imt):
        imtU0 = str(imt).upper()
        if imtU0 == "MMI":
            return measurement_sigma_mmi
        if imtU0 == "PGA":
            return measurement_sigma_pga
        return 0.30

    # ============================================================
    # B) LOCAL MULTI-TARGET PLOTS: two Taiwan cities (points)
    # ============================================================

    # Taipei:   25.0330, 121.5654
    # Kaohsiung:22.6273, 120.3014


    points_tw = [
        {"id": "Taipei",        "lat": 25.0330, "lon": 121.5654},
        {"id": "Kaohsiung",     "lat": 22.6273, "lon": 120.3014},
        {"id": "Hualien",       "lat": 23.9872, "lon": 121.6015},
        {"id": "TW_Point_01",   "lat": 24.9770, "lon": 120.9240},  # random coordinate
    ]


    # ---- 1) Uncertainty decay at each city (total sigma) + combined overlay ----
    df_tw_sigma = shake.uq_plot_targets_decay(
        version_list=version_list_int,
        imt=imtU,
        points=points_tw,
        areas=None,
        what="sigma_total",
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        agg="mean",
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        figsize=figsize,
        combined_figsize=combined_figsize,
        dpi=dpi,
        xrotation=xrotation,
        legend_kwargs=legend_kwargs,

        # NEW sizing + labels
        xlabel="ShakeMap version",
        ylabel=f"σ_total ({imtU})",
        title_size=title_size,
        label_size=label_size,
        tick_size=tick_size,
        legend_size=legend_size,
        line_kwargs=line_kwargs,
        grid_kwargs=grid_kwargs,

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,
        plot_combined=True,
        title=f"{imtU} uncertainty decay @ {{target}} (sigma_total)",
    )

    # ---- 2) Mean + sigma panels at each city (3-panel per target) ----
    df_tw_panels = shake.uq_plot_targets_mean_and_sigma(
        version_list=version_list_int,
        imt=imtU,
        points=points_tw,
        areas=None,
        methods=("ShakeMap", "bayes", "hierarchical", "kriging", "montecarlo"),
        agg="mean",
        prior_version=v0,

        sigma_total_from_shakemap=sigma_total_from_shakemap,
        sigma_aleatory=sigma_aleatory_for_imt(imtU),

        update_radius_km=update_radius_km,
        kernel=kernel,
        kernel_scale_km=kernel_scale_km,
        measurement_sigma=measurement_sigma_for_imt(imtU),

        ok_range_km=ok_range_km,
        ok_variogram=ok_variogram,
        ok_nugget=ok_nugget,
        ok_sill=ok_sill,
        ok_cap_sigma_to_prior=ok_cap_sigma_to_prior,

        mc_nsim=mc_nsim,
        mc_include_aleatory=mc_include_aleatory,

        figsize=(24, 18),
        dpi=dpi,
        xrotation=xrotation,
        legend_kwargs=legend_kwargs,

        # NEW sizing + labels
        xlabel="ShakeMap version",
        ylabel_mean=f"{imtU}",
        ylabel_sigma_total=f"σ_total ({imtU})",
        ylabel_sigma_epistemic=f"σ_ep ({imtU})",
        title_size=title_size,
        label_size=label_size,
        tick_size=tick_size,
        legend_size=legend_size,
        line_kwargs=line_kwargs,
        grid_kwargs=grid_kwargs,

        output_path=out_root,
        save=True,
        save_formats=save_formats,
        show=True,
        plot_combined=True,
        title="Target series @ {target} — {imt} (mean + σ_total + σ_ep)",
    )

    # ============================================================
    # Quick notes: changing settings/models
    # ============================================================
    # - Switch IMT: set imt_choice = "MMI" or "PGA"
    # - Make decay log-scale: pass ylog=True to uq_plot_targets_decay(...)
    # - Tighten/loosen observation influence: update_radius_km, kernel_scale_km
    # - Kriging smoothness: ok_range_km (bigger = smoother)
    # - Monte Carlo: mc_include_aleatory=False for epistemic-only sampling
    #
    print("[DONE] Local target patch demo complete for IMT =", imtU)
    print("Check outputs in:", out_root)


## Audit Station Filtering 

In [ ]:
if uncertainty_quantification:

    
    # ============================================================
    # SHAKEuq UQ sanity/audit helper
    # Run AFTER: uq_state = shake.uq_build_dataset(...)
    # ============================================================
    
    from pathlib import Path
    import json
    import numpy as np
    import pandas as pd
    
    def uq_station_filter_audit(shake, versions=None, out_dir=None, max_print_versions=5):
        """
        Produces:
          1) A summary table using shake.uq_state["sanity_rows"]
          2) A deeper per-version station parsing audit:
             - how many stations exist
             - how many got dropped by lon/lat coercion
             - how many got dropped by value filter
             - what columns existed (to detect PGA column name mismatch)
             - how many IDs/station_codes are missing (to diagnose plotting dedup bug)
        Optionally writes CSV/JSON into out_dir.
    
        Returns: (df_sanity, df_deep)
        """
        if not hasattr(shake, "uq_state") or shake.uq_state is None:
            raise RuntimeError("UQ dataset not built yet. Run shake.uq_build_dataset(...) first.")
    
        if versions is None:
            versions = [int(v) for v in shake.uq_state["version_list"]]
        else:
            versions = [int(v) for v in list(versions)]
    
        # ---------- 1) existing sanity table ----------
        sanity_rows = shake.uq_state.get("sanity_rows", [])
        df_sanity = pd.DataFrame(sanity_rows).sort_values("version")
    
        # ---------- 2) deeper audit ----------
        deep_rows = []
        for v in versions:
            raw = shake.uq_state["versions_raw"].get(int(v), {})
            dbg = raw.get("station_parse_debug", {}) or {}
            inst = raw.get("stations", {}).get("instrumented", []) or []
            dyfi = raw.get("stations", {}).get("dyfi", []) or []
    
            # Basic counts from parsed lists
            n_inst_parsed = len(inst)
            n_dyfi_parsed = len(dyfi)
    
            # How many instrumented have actual pga value vs only lon/lat (pga=None)
            n_inst_with_pga = sum(1 for o in inst if o.get("pga", None) is not None and np.isfinite(o.get("pga", np.nan)))
            n_inst_missing_pga = n_inst_parsed - n_inst_with_pga
    
            # % retention estimates from debug counters (if present)
            pga_df_rows = int(dbg.get("pga_df_rows", 0) or 0)
            pga_lonlat = int(dbg.get("pga_rows_after_lonlat_filter", 0) or 0)
            pga_val = int(dbg.get("pga_rows_after_value_filter", 0) or 0)
    
            mmi_df_rows = int(dbg.get("mmi_df_rows", 0) or 0)
            mmi_lonlat = int(dbg.get("mmi_rows_after_lonlat_filter", 0) or 0)
            mmi_val = int(dbg.get("mmi_rows_after_intensity_filter", 0) or 0)
    
            # Column diagnostics (to detect column name mismatch)
            pga_cols = dbg.get("pga_columns", [])
            mmi_cols = dbg.get("mmi_columns", [])
    
            # Heuristic flags: likely-too-harsh filtering
            # - Many rows exist, but few values survive.
            flag_pga_value_collapse = (pga_df_rows >= 50 and pga_val <= max(3, int(0.05 * pga_df_rows)))
            flag_mmi_value_collapse = (mmi_df_rows >= 50 and mmi_val <= max(3, int(0.05 * mmi_df_rows)))
    
            # - Many parsed instrumented points but many missing pga => column mismatch
            flag_pga_col_mismatch = (pga_df_rows >= 50 and n_inst_with_pga <= max(3, int(0.05 * pga_df_rows)))
    
            deep_rows.append({
                "version": int(v),
    
                # debug counter summary
                "pga_df_rows": pga_df_rows,
                "pga_rows_after_lonlat_filter": pga_lonlat,
                "pga_rows_after_value_filter": pga_val,
                "mmi_df_rows": mmi_df_rows,
                "mmi_rows_after_lonlat_filter": mmi_lonlat,
                "mmi_rows_after_intensity_filter": mmi_val,
    
                # parsed lists summary
                "n_instrumented_parsed": n_inst_parsed,
                "n_instrumented_with_pga_value": n_inst_with_pga,
                "n_instrumented_missing_pga_value": n_inst_missing_pga,
                "n_dyfi_parsed": n_dyfi_parsed,
    
                # diagnostic notes
                "pga_columns": "|".join([str(c) for c in (pga_cols or [])]),
                "mmi_columns": "|".join([str(c) for c in (mmi_cols or [])]),
                "station_parse_note": str(dbg.get("note", "")),
    
                "FLAG_pga_value_collapse": bool(flag_pga_value_collapse),
                "FLAG_mmi_value_collapse": bool(flag_mmi_value_collapse),
                "FLAG_pga_column_mismatch": bool(flag_pga_col_mismatch),
            })
    
        df_deep = pd.DataFrame(deep_rows).sort_values("version")
    
        # ---------- printing ----------
        print("\n====================")
        print("UQ SANITY SUMMARY")
        print("====================")
        keep_cols = [
            "version", "grid_shape",
            "n_instrumented", "n_dyfi",
            "uncertainty_xml_exists", "stationlist_exists",
            "pga_df_rows", "pga_rows_after_lonlat_filter", "pga_rows_after_value_filter",
            "mmi_df_rows", "mmi_rows_after_lonlat_filter", "mmi_rows_after_intensity_filter",
            "station_parse_note",
        ]
        keep_cols = [c for c in keep_cols if c in df_sanity.columns]
        print(df_sanity[keep_cols].to_string(index=False))
    
        print("\n====================")
        print("DEEP STATION FILTER AUDIT (key columns)")
        print("====================")
        key_cols = [
            "version",
            "pga_df_rows", "pga_rows_after_lonlat_filter", "pga_rows_after_value_filter",
            "n_instrumented_parsed", "n_instrumented_with_pga_value", "n_instrumented_missing_pga_value",
            "FLAG_pga_value_collapse", "FLAG_pga_column_mismatch",
            "mmi_df_rows", "mmi_rows_after_lonlat_filter", "mmi_rows_after_intensity_filter",
            "n_dyfi_parsed", "FLAG_mmi_value_collapse",
            "station_parse_note",
        ]
        key_cols = [c for c in key_cols if c in df_deep.columns]
        print(df_deep[key_cols].to_string(index=False))
    
        # Print the first few versions' column lists (useful for PGA column name mismatch)
        print("\n====================")
        print("COLUMN LISTS (first few versions)")
        print("====================")
        shown = 0
        for _, r in df_deep.iterrows():
            if shown >= max_print_versions:
                break
            v = int(r["version"])
            print(f"\n--- v{v} ---")
            print("PGA columns:", (r.get("pga_columns", "")[:400] + ("..." if len(str(r.get("pga_columns",""))) > 400 else "")))
            print("MMI columns:", (r.get("mmi_columns", "")[:400] + ("..." if len(str(r.get("mmi_columns",""))) > 400 else "")))
            shown += 1
    
        # ---------- optional export ----------
        if out_dir is not None:
            out_dir = Path(out_dir)
            out_dir.mkdir(parents=True, exist_ok=True)
    
            df_sanity.to_csv(out_dir / "uq_sanity_summary.csv", index=False)
            df_deep.to_csv(out_dir / "uq_station_filter_audit.csv", index=False)
    
            with open(out_dir / "uq_station_filter_audit.json", "w", encoding="utf-8") as f:
                json.dump({"sanity_rows": sanity_rows, "deep_rows": deep_rows}, f, indent=2)
    
            print(f"\n[OK] Wrote sanity outputs to: {out_dir}")
    
        return df_sanity, df_deep
    
    
    # ============================================================
    # Example usage (right after uq_build_dataset)
    # ============================================================
    
    # Choose where to write outputs (optional)
    audit_out_dir = Path("./export") / str(event_id) / "uq" / "audit"
    
    df_sanity, df_deep = uq_station_filter_audit(
        shake,
        versions=version_list_int,
        out_dir=audit_out_dir,
        max_print_versions=3,
    )
    
    # If you're in Jupyter, display tables nicely:
    try:
        display(df_sanity)
        display(df_deep)
    except Exception:
        pass


In [ ]:
# Location Tracking